In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as nc
from netCDF4 import Dataset
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
from numpy.polynomial.polynomial import polyfit
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from numpy import cov
import xarray as xr
import scipy.stats as stats
import math
from scipy.stats import linregress
import numpy.polynomial.polynomial as poly
from sklearn.linear_model import LinearRegression
import scipy.interpolate as interpolate
from scipy.optimize import curve_fit
import cartopy.io.img_tiles as cimgt
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.io.shapereader as shpreader
import cartopy.io.img_tiles as cimgt
import cartopy.mpl.geoaxes
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from cartopy import config
from matplotlib.transforms import offset_copy
%matplotlib inline
import cmocean
import glob
from scipy.io import netcdf_file as netcdf
from cartopy.feature import ShapelyFeature
from cartopy.io.shapereader import Reader
from shapely import geometry
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from scipy import integrate
import scipy as sci
import cartopy.io.shapereader as shpreader
from cartopy.feature import ShapelyFeature
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
from mpl_toolkits.mplot3d.axes3d import Axes3D
from mpl_toolkits.mplot3d import proj3d
from scipy.stats import kde
from scipy import signal
import metpy.calc as mpc
import matplotlib.pylab as pl
from metpy.units import units
import pymannkendall as mk

In [ ]:
plt.rcParams.update({'font.size': 7})
plt.rcParams['font.family'] = 'Arial'
mpl.rcParams['figure.dpi'] = 300
plt.rcParams['axes.xmargin'] = 0
plt.rcParams['axes.ymargin'] = 0

In [ ]:
def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

In [ ]:
ymax = 4.2
tmax = 6
# parameters from simulations
omega = 1.41e-4 #tidal frequency
T = 2*np.pi/omega 
nu = 2.0e-6 #molecular viscosity (m^2 s^-1)
Pr = 14 #Prandtl number (dimensionles)
L = 3.35e5 # Latent heat of melting (units)
cp = 4.184e3 # Heat capacity of the ocean (units)
sy = 365*24*3600
rho0 = 1030 #kg/m^{-3}
Sc = 1500
Rc = 0.2
Xi = 0.052
alpha = 3.8e-5
beta = 7.8e-4
corio = 1.37e-4
gravity = 9.8
vonk = 0.41
kT = 1.4e-7 #m^2/s Thermal Diffusivity
kS = 1.3e-9 #m^2/s Salt Diffusivity

In [ ]:
s_df = xr.open_dataset('C:/Users/bajus/Downloads/tidal_melting/tidal_melting/data/stokes/ISOBL_model_output.nc')

In [ ]:
se_df = xr.open_dataset('C:/Users/bajus/Downloads/tidal_melting/tidal_melting/data/stokes-ekman/ISOBL_model_output.nc')

In [ ]:
s_dUdy = s_df['U'].differentiate('y')
s_dWdy = s_df['W'].differentiate('y')

se_dUdy = se_df['U'].differentiate('y')
se_dWdy = se_df['W'].differentiate('y')

In [ ]:
s_S2 = s_dUdy**2 + s_dWdy**2
se_S2 = se_dUdy**2 + se_dWdy**2

In [ ]:
s_Ri = s_df['N2']/s_S2
se_Ri = se_df['N2']/se_S2

In [ ]:
dtdz_s = (s_df['Temp'][:,1]-s_df['Temp'][:,0])/(s_df['y'][1]-s_df['y'][0])
meltrate_s = (cp/L)*(nu/Pr)*(1/0.92)*dtdz_s*sy

dtdz_se = (se_df['Temp'][:,1]-se_df['Temp'][:,0])/(se_df['y'][1]-se_df['y'][0])
meltrate_se = (cp/L)*(nu/Pr)*(1/0.92)*dtdz_se*sy

In [ ]:
def ustar_calc(ubar, wbar, y, nu, rho):
            
    spd = (ubar**2+wbar**2)**0.5
    dUdy = (((ubar[:,2]-ubar[:,1])/(y[2]-y[1]))**2+((wbar[:,2]-wbar[:,1])/(y[2]-y[1]))**2)**0.5
    
    tau_w = rho*nu*(dUdy)
    ustar = (tau_w/rho)**0.5
    return ustar

In [ ]:
s_ustar = ustar_calc(s_df['U'].data, s_df['W'].data, s_df['y'].data, nu, rho0)
se_ustar = ustar_calc(se_df['U'].data, se_df['W'].data, se_df['y'].data, nu, rho0)

In [ ]:
s_ds = xr.open_dataset('C:/Users/bajus/Downloads/tidal_melting/tidal_melting/data/stokes/mean_prof_stat.nc')
se_ds = xr.open_dataset('C:/Users/bajus/Downloads/tidal_melting/tidal_melting/data/stokes-ekman/mean_prof_stat.nc')

In [ ]:
### remove duplicated y and t values - this is applied to all the variables in the dataset
_, index = np.unique(s_ds['y'], return_index=True)
s_dtmp = s_ds.isel(y=index)

_, index = np.unique(s_ds['t'], return_index=True)
s_dtmp = s_dtmp.isel(t=index)

### remove anything with a nan in coordinate "t" (time)
idx = np.squeeze(np.argwhere(~np.isnan(s_dtmp.t.values)))
s_dtmp = s_dtmp.isel(t=idx)

In [ ]:
nhours = ((s_dtmp.t[-2]-s_dtmp.t[1])/3600)
NT = len(s_ds.t)
dT_minutes = 10
NT_light = int(nhours.round()*(60/dT_minutes))
if NT>NT_light:
    print('yep, sub-sampling in t')
    t_light = np.linspace(s_dtmp.t[1],s_dtmp.t[-2],NT_light)
    s_dtmp = s_dtmp.sel(t=t_light, method='nearest')

In [ ]:
## domain unnecessarily deep for some of my early simulations. Restrict to 10 m
ymax = 10
if s_dtmp.y[-2]>ymax:
    s_dtmp = s_dtmp.sel(y=slice(None, 10.05))
    print('yep, sub-sampling in y')

In [ ]:
### remove duplicated y and t values - this is applied to all the variables in the dataset
_, index = np.unique(se_ds['y'], return_index=True)
se_dtmp = se_ds.isel(y=index)

_, index = np.unique(se_ds['t'], return_index=True)
se_dtmp = se_dtmp.isel(t=index)

### remove anything with a nan in coordinate "t" (time)
idx = np.squeeze(np.argwhere(~np.isnan(se_dtmp.t.values)))
se_dtmp = se_dtmp.isel(t=idx)

In [ ]:
nhours = ((se_dtmp.t[-2]-se_dtmp.t[1])/3600)
NT = len(se_ds.t)
dT_minutes = 10
NT_light = int(nhours.round()*(60/dT_minutes))
if NT>NT_light:
    print('yep, sub-sampling in t')
    t_light = np.linspace(se_dtmp.t[1],se_dtmp.t[-2],NT_light)
    se_dtmp = se_dtmp.sel(t=t_light, method='nearest')

In [ ]:
## domain unnecessarily deep for some of my early simulations. Restrict to 10 m
ymax = 10
if se_dtmp.y[-2]>ymax:
    se_dtmp = se_dtmp.sel(y=slice(None, 10.05))
    print('yep, sub-sampling in y')

In [ ]:
s_u_star = (s_dtmp['uv']**2 + s_dtmp['wv']**2)**(1/4)
se_u_star = (se_dtmp['uv']**2 + se_dtmp['wv']**2)**(1/4)

In [ ]:
s_speed = (s_df['U']**2 + s_df['W']**2)**(1/2)
se_speed = (se_df['U']**2 + se_df['W']**2)**(1/2)

In [ ]:
def cycle_average_2d(var, time, y, omega, start_cycle, end_cycle, dt):
    import numpy as np
    
    T = 2*np.pi/omega
    NT = int(np.floor(T/dt))
    print(NT)
    t_cycle = np.linspace(0, T, NT)
    
    if np.floor(time[-2]/T)<end_cycle:
        print('record not', end_cycle, ' cycles long!!')
        return
    
    var_cycle = np.zeros((len(t_cycle), len(y)))
    
    for ii in range(start_cycle,end_cycle):
        ttmp = time - ii*T
        for jj in range(len(y)):
            tmp = np.interp(t_cycle, ttmp, var[:,jj])
            var_cycle[:,jj] = var_cycle[:,jj]+tmp
        
    var_cycle = var_cycle/(end_cycle-start_cycle)
    return t_cycle, var_cycle

In [ ]:
s_S2_1c = cycle_average_2d(s_S2, s_df['t'], s_df['y'], omega, 3, 6, 100)
se_S2_1c = cycle_average_2d(se_S2, se_df['t'], se_df['y'], omega, 3, 6, 100)

In [ ]:
s_N2_1c = cycle_average_2d(s_df['N2'], s_df['t'], s_df['y'], omega, 3, 6, 100)
se_N2_1c = cycle_average_2d(se_df['N2'], se_df['t'], se_df['y'], omega, 3, 6, 100)

In [ ]:
s_u_star_1c = cycle_average_2d(s_u_star, s_df['t'], s_df['y'], omega, 3, 6, 100)
se_u_star_1c = cycle_average_2d(se_u_star, se_df['t'], se_df['y'], omega, 3, 6, 100)

In [ ]:
s_TKE_1c = cycle_average_2d(s_df['TKE'], s_df['t'], s_df['y'], omega, 3, 6, 100)
se_TKE_1c = cycle_average_2d(se_df['TKE'], se_df['t'], se_df['y'], omega, 3, 6, 100)

In [ ]:
s_speed_1c = cycle_average_2d(s_speed, s_df['t'], s_df['y'], omega, 3, 6, 100)
se_speed_1c = cycle_average_2d(se_speed, se_df['t'], se_df['y'], omega, 3, 6, 100)

In [ ]:
s_Ri_1c = cycle_average_2d(s_Ri, s_df['t'], s_df['y'], omega, 3, 6, 100)
se_Ri_1c = cycle_average_2d(se_Ri, se_df['t'], se_df['y'], omega, 3, 6, 100)

In [ ]:
len(s_TKE_1c[0][:223])

In [ ]:
s_S2_1c[0]/T

In [ ]:
fig, (ax2, ax1) = plt.subplots(ncols=2,nrows=1,figsize=(8,3))


cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_S2_1c[0])):
    ax1.plot(se_S2_1c[1][i,:],se_df['y'],linewidth=0.25,color=cmap(i/len(s_S2_1c[0])))

    
m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,1)
cbar1 = plt.colorbar(m1, ax=ax1, boundaries=np.linspace(0,1,len(s_S2_1c[0])),ticks=[0,0.2,0.4,0.6,0.8,1]
                     ,pad=0.05,orientation='vertical')

ax1.set_ylim(5,0)
ax1.set_xlim(0,5e-5)


ax1.yaxis.set_minor_locator(MultipleLocator(0.2))
ax1.xaxis.set_minor_locator(MultipleLocator(0.2e-5))

ax1.set_title(r'S$^{2}$: Stokes-Ekman')





cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_S2_1c[0])):
    ax2.plot(s_S2_1c[1][i,:],s_df['y'],linewidth=0.25,color=cmap(i/len(s_S2_1c[0])))

    
m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,1)
cbar1 = plt.colorbar(m1, ax=ax2, boundaries=np.linspace(0,1,len(s_S2_1c[0])),ticks=[0,0.2,0.4,0.6,0.8,1]
                     ,pad=0.05,orientation='vertical')

ax2.set_ylim(5,0)
ax2.set_xlim(0,5e-5)


ax2.yaxis.set_minor_locator(MultipleLocator(0.2))
ax2.xaxis.set_minor_locator(MultipleLocator(0.2e-5))

ax2.set_title(r'S$^{2}$: Stokes')



plt.tight_layout()
plt.show()

In [ ]:
fig, (ax2, ax1) = plt.subplots(ncols=2,nrows=1,figsize=(8,3))


cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_S2_1c[0][:223])):
    ax1.plot(se_S2_1c[1][:223,:][i,:],se_df['y'],linewidth=0.25,color=cmap(i/len(s_S2_1c[0][:223])))

    
m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,180)
cbar1 = plt.colorbar(m1, ax=ax1, boundaries=np.linspace(0,180,len(s_S2_1c[0][:223])),ticks=[0,45,90,135,180]
                     ,pad=0.05,orientation='vertical')

ax1.set_ylim(5,0)
ax1.set_xlim(0,5e-5)


ax1.yaxis.set_minor_locator(MultipleLocator(0.2))
ax1.xaxis.set_minor_locator(MultipleLocator(0.2e-5))

ax1.set_title(r'S$^{2}$: Stokes-Ekman')





cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_S2_1c[0][:223])):
    ax2.plot(s_S2_1c[1][:223,:][i,:],s_df['y'],linewidth=0.25,color=cmap(i/len(s_S2_1c[0][:223])))

    
mm1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,180)
cbar1 = plt.colorbar(m1, ax=ax2, boundaries=np.linspace(0,180,len(s_S2_1c[0][:223])),ticks=[0,45,90,135,180]
                     ,pad=0.05,orientation='vertical')
ax2.set_ylim(5,0)
ax2.set_xlim(0,5e-5)


ax2.yaxis.set_minor_locator(MultipleLocator(0.2))
ax2.xaxis.set_minor_locator(MultipleLocator(0.2e-5))

ax2.set_title(r'S$^{2}$: Stokes')



plt.tight_layout()
plt.show()

In [ ]:
fig, (ax2, ax1) = plt.subplots(ncols=2,nrows=1,figsize=(8,3))


cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_TKE_1c[0])):
    ax1.plot(se_TKE_1c[1][i,:],se_df['y'],linewidth=0.25,color=cmap(i/len(s_TKE_1c[0])))

    
m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,1)
cbar1 = plt.colorbar(m1, ax=ax1, boundaries=np.linspace(0,1,len(s_TKE_1c[0])),ticks=[0,0.2,0.4,0.6,0.8,1]
                     ,pad=0.05,orientation='vertical')

ax1.set_ylim(5,0)
ax1.set_xlim(0,5e-6)


ax1.yaxis.set_minor_locator(MultipleLocator(0.2))
ax1.xaxis.set_minor_locator(MultipleLocator(0.2e-5))

ax1.set_title(r'TKE: Stokes-Ekman')





cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_TKE_1c[0])):
    ax2.plot(s_TKE_1c[1][i,:],s_df['y'],linewidth=0.25,color=cmap(i/len(s_TKE_1c[0])))

    
m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,1)
cbar1 = plt.colorbar(m1, ax=ax2, boundaries=np.linspace(0,1,len(s_TKE_1c[0])),ticks=[0,0.2,0.4,0.6,0.8,1]
                     ,pad=0.05,orientation='vertical')

ax2.set_ylim(5,0)
ax1.set_xlim(0,5e-6)


ax2.yaxis.set_minor_locator(MultipleLocator(0.2))
ax2.xaxis.set_minor_locator(MultipleLocator(0.2e-5))

ax2.set_title(r'TKE: Stokes')



plt.tight_layout()
plt.show()

In [ ]:
fig, (ax2, ax1) = plt.subplots(ncols=2,nrows=1,figsize=(8,3))


cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_TKE_1c[0][:223])):
    ax1.plot(se_TKE_1c[1][:223,:][i,:],se_df['y'],linewidth=0.25,color=cmap(i/len(s_TKE_1c[0][:223])))

    
m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,180)
cbar1 = plt.colorbar(m1, ax=ax1, boundaries=np.linspace(0,180,len(s_S2_1c[0][:223])),ticks=[0,45,90,135,180]
                     ,pad=0.05,orientation='vertical')

ax1.set_ylim(5,0)
ax1.set_xlim(0,5e-6)


ax1.yaxis.set_minor_locator(MultipleLocator(0.2))
ax1.xaxis.set_minor_locator(MultipleLocator(0.2e-5))

ax1.set_title(r'TKE: Stokes-Ekman')





cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_TKE_1c[0][:223])):
    ax2.plot(s_TKE_1c[1][:223,:][i,:],s_df['y'],linewidth=0.25,color=cmap(i/len(s_TKE_1c[0][:223])))

    
m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,180)
cbar1 = plt.colorbar(m1, ax=ax2, boundaries=np.linspace(0,180,len(s_S2_1c[0][:223])),ticks=[0,45,90,135,180]
                     ,pad=0.05,orientation='vertical')
ax2.set_ylim(5,0)
ax1.set_xlim(0,5e-6)


ax2.yaxis.set_minor_locator(MultipleLocator(0.2))
ax2.xaxis.set_minor_locator(MultipleLocator(0.2e-5))

ax2.set_title(r'TKE: Stokes')



plt.tight_layout()
plt.show()

In [ ]:
fig, (ax2, ax1) = plt.subplots(ncols=2,nrows=1,figsize=(8,3))


cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_speed_1c[0])):
    ax1.plot(se_speed_1c[1][i,:],se_df['y'],linewidth=0.25,color=cmap(i/len(s_S2_1c[0])))

    
m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,1)
cbar1 = plt.colorbar(m1, ax=ax1, boundaries=np.linspace(0,1,len(s_S2_1c[0])),ticks=[0,0.2,0.4,0.6,0.8,1]
                     ,pad=0.05,orientation='vertical')

ax1.set_ylim(5,0)
ax1.set_xlim(0,0.03)


ax1.yaxis.set_minor_locator(MultipleLocator(0.2))
#ax1.xaxis.set_minor_locator(MultipleLocator(0.2e-5))

ax1.set_title(r'Speed: Stokes-Ekman')





cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_speed_1c[0])):
    ax2.plot(s_speed_1c[1][i,:],s_df['y'],linewidth=0.25,color=cmap(i/len(s_S2_1c[0])))

    
m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,1)
cbar1 = plt.colorbar(m1, ax=ax2, boundaries=np.linspace(0,1,len(s_S2_1c[0])),ticks=[0,0.2,0.4,0.6,0.8,1]
                     ,pad=0.05,orientation='vertical')

ax2.set_ylim(5,0)
ax2.set_xlim(0,0.03)


ax2.yaxis.set_minor_locator(MultipleLocator(0.2))
#ax2.xaxis.set_minor_locator(MultipleLocator(0.2e-5))

ax2.set_title(r'Speed: Stokes')



plt.tight_layout()
plt.show()

In [ ]:
fig, (ax2, ax1) = plt.subplots(ncols=2,nrows=1,figsize=(8,3))


cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_speed_1c[0][:223])):
    ax1.plot(se_speed_1c[1][:223,:][i,:],se_df['y'],linewidth=0.25,color=cmap(i/len(s_S2_1c[0][:223])))

    
m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,180)
cbar1 = plt.colorbar(m1, ax=ax1, boundaries=np.linspace(0,180,len(s_S2_1c[0][:223])),ticks=[0,45,90,135,180]
                     ,pad=0.05,orientation='vertical')

ax1.set_ylim(5,0)
ax1.set_xlim(0,0.03)


ax1.yaxis.set_minor_locator(MultipleLocator(0.2))
#ax1.xaxis.set_minor_locator(MultipleLocator(0.2e-5))

ax1.set_title(r'Speed: Stokes-Ekman')





cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_speed_1c[0][:223])):
    ax2.plot(s_speed_1c[1][:223,:][i,:],s_df['y'],linewidth=0.25,color=cmap(i/len(s_S2_1c[0][:223])))

    
m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,180)
cbar1 = plt.colorbar(m1, ax=ax2, boundaries=np.linspace(0,180,len(s_S2_1c[0][:223])),ticks=[0,45,90,135,180]
                     ,pad=0.05,orientation='vertical')

ax2.set_ylim(5,0)
ax2.set_xlim(0,0.03)


ax2.yaxis.set_minor_locator(MultipleLocator(0.2))
#ax2.xaxis.set_minor_locator(MultipleLocator(0.2e-5))

ax2.set_title(r'Speed: Stokes')



plt.tight_layout()
plt.show()

In [ ]:
fig, (ax2, ax1) = plt.subplots(ncols=2,nrows=1,figsize=(8,3))


cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_Ri_1c[0])):
    ax1.plot(se_Ri_1c[1][i,:],se_df['y'],linewidth=0.25,color=cmap(i/len(s_Ri_1c[0])))

    
m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,1)
cbar1 = plt.colorbar(m1, ax=ax1, boundaries=np.linspace(0,1,len(s_Ri_1c[0])),ticks=[0,0.2,0.4,0.6,0.8,1]
                     ,pad=0.05,orientation='vertical')

ax1.set_ylim(5,0)
ax1.set_xlim(0,0.4)


ax1.yaxis.set_minor_locator(MultipleLocator(0.2))
ax1.xaxis.set_minor_locator(MultipleLocator(0.1))

ax1.set_title(r'R$_{g}$: Stokes-Ekman')





cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_Ri_1c[0])):
    ax2.plot(s_Ri_1c[1][i,:],s_df['y'],linewidth=0.25,color=cmap(i/len(s_Ri_1c[0])))

    
m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,1)
cbar1 = plt.colorbar(m1, ax=ax2, boundaries=np.linspace(0,1,len(s_Ri_1c[0])),ticks=[0,0.2,0.4,0.6,0.8,1]
                     ,pad=0.05,orientation='vertical')

ax2.set_ylim(5,0)
ax2.set_xlim(0,0.4)


ax2.yaxis.set_minor_locator(MultipleLocator(0.2))
ax2.xaxis.set_minor_locator(MultipleLocator(0.1))

ax2.set_title(r'R$_{g}$: Stokes')



plt.tight_layout()
plt.show()

In [ ]:
fig, (ax2, ax1) = plt.subplots(ncols=2,nrows=1,figsize=(8,3))


cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_Ri_1c[0][:223])):
    ax1.plot(se_Ri_1c[1][:223,:][i,:],se_df['y'],linewidth=0.25,color=cmap(i/len(s_Ri_1c[0][:223])))

    
m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,180)
cbar1 = plt.colorbar(m1, ax=ax1, boundaries=np.linspace(0,180,len(s_S2_1c[0][:223])),ticks=[0,45,90,135,180]
                     ,pad=0.05,orientation='vertical')

ax1.set_ylim(5,0)
ax1.set_xlim(0,0.5)


ax1.yaxis.set_minor_locator(MultipleLocator(0.2))
ax1.xaxis.set_minor_locator(MultipleLocator(0.1))

ax1.set_title(r'R$_{g}$: Stokes-Ekman')





cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_Ri_1c[0][:223])):
    ax2.plot(s_Ri_1c[1][:223,:][i,:],s_df['y'],linewidth=0.25,color=cmap(i/len(s_Ri_1c[0][:223])))

    
m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,180)
cbar1 = plt.colorbar(m1, ax=ax2, boundaries=np.linspace(0,180,len(s_S2_1c[0][:223])),ticks=[0,45,90,135,180]
                     ,pad=0.05,orientation='vertical')

ax2.set_ylim(5,0)
ax2.set_xlim(0,0.5)


ax2.yaxis.set_minor_locator(MultipleLocator(0.2))
ax2.xaxis.set_minor_locator(MultipleLocator(0.1))

ax2.set_title(r'R$_{g}$: Stokes')



plt.tight_layout()
plt.show()

In [ ]:
s_turner = (180/math.pi)*np.arctan2(-alpha*(np.diff(s_df['Temp'],axis=1)/np.diff(s_df['y'])) - beta*(np.diff(s_df['Sal'],axis=1)/np.diff(s_df['y'])),-alpha*(np.diff(s_df['Temp'],axis=1)/np.diff(s_df['y'])) + beta*(np.diff(s_df['Sal'],axis=1)/np.diff(s_df['y'])))
se_turner = (180/math.pi)*np.arctan2(-alpha*(np.diff(se_df['Temp'],axis=1)/np.diff(se_df['y'])) - beta*(np.diff(se_df['Sal'],axis=1)/np.diff(se_df['y'])),-alpha*(np.diff(se_df['Temp'],axis=1)/np.diff(se_df['y'])) + beta*(np.diff(se_df['Sal'],axis=1)/np.diff(se_df['y'])))


In [ ]:
s_turner = (180/math.pi)*np.arctan2(((-alpha*s_df['Temp'].differentiate('y')) - (beta*s_df['Sal'].differentiate('y'))),((-alpha*s_df['Temp'].differentiate('y')) + (beta*s_df['Sal'].differentiate('y'))) )
se_turner = (180/math.pi)*np.arctan2(((-alpha*se_df['Temp'].differentiate('y')) - (beta*se_df['Sal'].differentiate('y'))),((-alpha*se_df['Temp'].differentiate('y')) + (beta*se_df['Sal'].differentiate('y'))) )


In [ ]:
np.shape(s_turner)

In [ ]:
s_turner_1c = cycle_average_2d(s_turner, s_df['t'], s_df['y'][:], omega, 3, 6, 100)
se_turner_1c = cycle_average_2d(se_turner, se_df['t'], se_df['y'][:], omega, 3, 6, 100)

In [ ]:
s_turner_1c[1]

In [ ]:
fig, (ax2, ax1) = plt.subplots(ncols=2,nrows=1,figsize=(8,3))


cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_turner_1c[0][:223])):
    ax1.plot(se_turner_1c[1][:223,:][i,:],se_df['y'][:],linewidth=0.25,color=cmap(i/len(s_turner_1c[0][:][:223])))

    
m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,1)
cbar1 = plt.colorbar(m1, ax=ax1, boundaries=np.linspace(0,1,len(s_turner_1c[0][:])),ticks=[0,0.2,0.4,0.6,0.8,1]
                     ,pad=0.05,orientation='vertical')

ax1.set_ylim(5,0)
ax1.set_xlim(-90,90)


ax1.yaxis.set_minor_locator(MultipleLocator(0.2))
ax1.xaxis.set_minor_locator(MultipleLocator(5))
ax1.xaxis.set_major_locator(MultipleLocator(45))

ax1.set_title(r'Turner Angle: Stokes-Ekman')





cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_turner_1c[0][:][:223])):
    ax2.plot(s_turner_1c[1][:223,:][i,:],s_df['y'][:],linewidth=0.25,color=cmap(i/len(s_turner_1c[0][:][:223])))

    
m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,1)
cbar1 = plt.colorbar(m1, ax=ax2, boundaries=np.linspace(0,1,len(s_turner_1c[0][:])),ticks=[0,0.2,0.4,0.6,0.8,1]
                     ,pad=0.05,orientation='vertical')

ax2.set_ylim(5,0)
ax2.set_xlim(-90,90)


ax2.yaxis.set_minor_locator(MultipleLocator(0.2))
ax2.xaxis.set_minor_locator(MultipleLocator(5))
ax2.xaxis.set_major_locator(MultipleLocator(45))

ax2.set_title(r'Turner Angle: Stokes')



plt.tight_layout()
plt.show()

In [ ]:
se_nturn = sci.ndimage.gaussian_filter1d(se_turner_1c[1],sigma=7,axis=1)
s_nturn = sci.ndimage.gaussian_filter1d(s_turner_1c[1],sigma=7,axis=1)

In [ ]:
fig, (ax2, ax1) = plt.subplots(ncols=2,nrows=1,figsize=(8,3))


cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_turner_1c[0][:223])):
    ax1.plot(se_nturn[:223,:][i,:],se_df['y'][:],linewidth=0.25,color=cmap(i/len(s_turner_1c[0][:223])))

    
m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,180)
cbar1 = plt.colorbar(m1, ax=ax1, boundaries=np.linspace(0,180,len(s_S2_1c[0][:223])),ticks=[0,45,90,135,180]
                     ,pad=0.05,orientation='vertical')

ax1.set_ylim(5,0)
ax1.set_xlim(-60,0)


ax1.yaxis.set_minor_locator(MultipleLocator(0.2))
ax1.xaxis.set_minor_locator(MultipleLocator(2))
ax1.xaxis.set_major_locator(MultipleLocator(10))

ax1.set_title(r'Turner Angle: Stokes-Ekman')

ax1.axvline(-45,linewidth=0.75,color='black')




cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_turner_1c[0][:223])):
    ax2.plot(s_nturn[:223,:][i,:],s_df['y'][:],linewidth=0.25,color=cmap(i/len(s_turner_1c[0][:223])))

    
m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,180)
cbar1 = plt.colorbar(m1, ax=ax2, boundaries=np.linspace(0,180,len(s_S2_1c[0][:223])),ticks=[0,45,90,135,180]
                     ,pad=0.05,orientation='vertical')

ax2.set_ylim(5,0)
ax2.set_xlim(-60,0)


ax2.yaxis.set_minor_locator(MultipleLocator(0.2))
ax2.xaxis.set_minor_locator(MultipleLocator(2))
ax2.xaxis.set_major_locator(MultipleLocator(10))

ax2.set_title(r'Turner Angle: Stokes')

ax2.axvline(-45,linewidth=0.75,color='black')

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.colors as mcolors

colors1 = plt.cm.Blues_r(np.linspace(0, 1, 3)[:])
colors11 = plt.cm.Greens(np.linspace(0, 1, 3)[:])

colorsw = plt.cm.Greens(np.linspace(0, 1, 3)[0])

colors2 = plt.cm.gist_heat(np.linspace(0, 1, 4)[0])
colors3 = plt.cm.gist_heat(np.linspace(0, 1, 4)[0])

colors_all = np.vstack((colors3,colors2,colors3,colors2,colors1,colorsw,colorsw,colors11,colors2,colors3,colors3,colors2))
mymap2 = mcolors.LinearSegmentedColormap.from_list('my_colormap', colors_all)

In [ ]:
plt.figure(figsize=(5,1.25))

plt.pcolormesh(s_turner_1c[0]/T,s_df['y'][:],s_nturn.transpose(),cmap=mymap2,vmin=-180,vmax=180)
plt.colorbar(ticks=[-90,-45,0,45,90])

plt.ylim(5,0)

plt.show()

In [ ]:
plt.figure(figsize=(5,1.25))

plt.pcolormesh(se_turner_1c[0]/T,se_df['y'][:],se_nturn.transpose(),cmap=mymap2,vmin=-180,vmax=180)
plt.colorbar(ticks=[-90,-45,0,45,90])

plt.ylim(5,0)

plt.show()

In [ ]:
se_turner_1c[1].shape

In [ ]:
445/5

In [ ]:
se_df

In [ ]:
s_n_bprod = 9.8*(s_df['VpTp']*alpha + s_df['VpSp']*beta)
se_n_bprod = 9.8*(se_df['VpTp']*alpha + se_df['VpSp']*beta)

In [ ]:
s_n_bprod_1c = cycle_average_2d(s_n_bprod, s_df['t'], s_df['y'][:], omega, 3, 6, 100)
se_n_bprod_1c = cycle_average_2d(se_n_bprod, se_df['t'], se_df['y'][:], omega, 3, 6, 100)

In [ ]:
fig = plt.figure(figsize=(6,5))

gs = fig.add_gridspec(4,3,width_ratios=[10,10,0.5])

ax1 = fig.add_subplot(gs[0:2,1])
ax2 = fig.add_subplot(gs[0:2,0])
ax3 = fig.add_subplot(gs[2:,0])
ax4 = fig.add_subplot(gs[2:,1])


cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(se_n_bprod_1c[0][:223])):
    ax1.plot(se_n_bprod_1c[1][:223,:][i,:],se_df['y'][:],linewidth=0.25,color=cmap(i/len(se_n_bprod_1c[0][:223])))

    
ax1.set_ylim(5,0)
ax1.set_xlim(-2e-10,2e-10)


ax1.yaxis.set_minor_locator(MultipleLocator(0.2))
ax1.xaxis.set_minor_locator(MultipleLocator(0.1e-10))
#ax1.xaxis.set_major_locator(MultipleLocator(10))

ax1.set_title(r'Bprod: Stokes-Ekman')

ax1.axvline(0,linewidth=0.75,color='black')




cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_n_bprod_1c[0][:223])):
    ax2.plot(s_n_bprod_1c[1][:223,:][i,:],s_df['y'][:],linewidth=0.25,color=cmap(i/len(s_n_bprod_1c[0][:223])))

    
ax2.set_ylim(5,0)
ax2.set_xlim(-2e-10,2e-10)


ax2.yaxis.set_minor_locator(MultipleLocator(0.2))
ax2.xaxis.set_minor_locator(MultipleLocator(0.1e-10))
#ax2.xaxis.set_major_locator(MultipleLocator(10))

#ax2.set_title(r'Bprod: Stokes')

ax2.axvline(0,linewidth=0.75,color='black')







cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_turner_1c[0][:223])):
    ax4.plot(se_nturn[:223,:][i,:],se_df['y'][:],linewidth=0.25,color=cmap(i/len(s_turner_1c[0][:223])))

    
ax4.set_ylim(5,0)
ax4.set_xlim(-60,0)


ax4.yaxis.set_minor_locator(MultipleLocator(0.2))
ax4.xaxis.set_minor_locator(MultipleLocator(2))
ax4.xaxis.set_major_locator(MultipleLocator(10))

#ax4.set_title(r'Turner Angle: Stokes-Ekman')

ax4.axvline(-45,linewidth=0.75,color='black')

ax4.annotate(r'DDC',xy=(-55.5,4.6),fontsize=9,color='black')
ax4.annotate(r'Stably Stratified',xy=(-31,4.6),fontsize=9,color='black')




cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_turner_1c[0][:223])):
    ax3.plot(s_nturn[:223,:][i,:],s_df['y'][:],linewidth=0.25,color=cmap(i/len(s_turner_1c[0][:223])))

    
ax3.set_ylim(5,0)
ax3.set_xlim(-60,0)


ax3.yaxis.set_minor_locator(MultipleLocator(0.2))
ax3.xaxis.set_minor_locator(MultipleLocator(2))
ax3.xaxis.set_major_locator(MultipleLocator(10))

#ax3.set_title(r'Turner Angle: Stokes')

ax3.axvline(-45,linewidth=0.75,color='black')

ax3.annotate(r'DDC',xy=(-55.5,4.6),fontsize=9,color='black')
ax3.annotate(r'Stably Stratified',xy=(-31,4.6),fontsize=9,color='black')

                      
                      
cax = fig.add_subplot(gs[1:3,2])

m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,180)
cbar1 = plt.colorbar(m1, cax=cax, boundaries=np.linspace(0,180,len(s_S2_1c[0][:223])),ticks=[0,45,90,135,180]
                     ,pad=0.05,orientation='vertical')
cbar1.ax.set_yticklabels(['0','$\pi$/4','$\pi$/2','3$\pi$/4','$\pi$'])



ax1.set_title(r'Rotating')

ax2.set_title(r'Non-Rotating')

ax2.set_ylabel('Buoyancy Production',fontsize=8)
ax3.set_ylabel('Turner Angle',fontsize=8)



plt.tight_layout()
plt.show()

In [ ]:
s_nturn.shape

In [ ]:
s_dc_array = np.nan*np.zeros((445,430))

for i in range(0,445,1):
    for j in range(0,430,1):
        if ((s_nturn[i,j] < -45) & (s_n_bprod_1c[1][i,j] < 0)):
            s_dc_array[i,j] = 1
            
        elif ((s_nturn[i,j] < -45) & (s_n_bprod_1c[1][i,j] >= 0)):
            s_dc_array[i,j] = 2
            
        elif ((s_nturn[i,j] >= -45) & (s_n_bprod_1c[1][i,j] < 0)):
            s_dc_array[i,j] = 3

        elif ((s_nturn[i,j] >= -45) & (s_n_bprod_1c[1][i,j] >= 0)):
            s_dc_array[i,j] = 4

In [ ]:
se_dc_array = np.nan*np.zeros((445,430))

for i in range(0,445,1):
    for j in range(0,430,1):
        if ((se_nturn[i,j] < -45) & (se_n_bprod_1c[1][i,j] < 0)):
            se_dc_array[i,j] = 1
            
        elif ((se_nturn[i,j] < -45) & (se_n_bprod_1c[1][i,j] >= 0)):
            se_dc_array[i,j] = 2
            
        elif ((se_nturn[i,j] >= -45) & (se_n_bprod_1c[1][i,j] < 0)):
            se_dc_array[i,j] = 3

        elif ((se_nturn[i,j] >= -45) & (se_n_bprod_1c[1][i,j] >= 0)):
            se_dc_array[i,j] = 4

In [ ]:
plt.figure(figsize=(4,1.5))

CS = plt.contourf(s_n_bprod_1c[0]/T,s_df['y'],s_dc_array.transpose(),levels=[0.5,1.5,2.5,3.5,4.5],cmap='rainbow')
plt.colorbar(CS)

plt.ylim(5,0)

plt.show()

In [ ]:
plt.figure(figsize=(4,1.5))

CS = plt.contourf(se_n_bprod_1c[0]/T,se_df['y'],se_dc_array.transpose(),levels=[0.5,1.5,2.5,3.5,4.5],cmap='rainbow')
plt.colorbar(CS)

plt.ylim(5,0)

plt.show()

In [ ]:
s_df

In [ ]:
s_diss_1c = cycle_average_2d(s_df['Dissipation'], s_df['t'], s_df['y'][:], omega, 3, 6, 100)
se_diss_1c = cycle_average_2d(se_df['Dissipation'], se_df['t'], se_df['y'][:], omega, 3, 6, 100)

In [ ]:
s_sprod_1c = cycle_average_2d(s_df['Shear_production'], s_df['t'], s_df['y'][:], omega, 3, 6, 100)
se_sprod_1c = cycle_average_2d(se_df['Shear_production'], se_df['t'], se_df['y'][:], omega, 3, 6, 100)

In [ ]:
fig, (ax2, ax1) = plt.subplots(ncols=2,nrows=1,figsize=(8,3))


cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(se_sprod_1c[0][:223])):
    ax1.plot(se_sprod_1c[1][:223,:][i,:],se_df['y'][:],linewidth=0.25,color=cmap(i/len(se_sprod_1c[0][:223])))

    
m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,180)
cbar1 = plt.colorbar(m1, ax=ax1, boundaries=np.linspace(0,180,len(s_S2_1c[0][:223])),ticks=[0,45,90,135,180]
                     ,pad=0.05,orientation='vertical')

ax1.set_ylim(5,0)
#ax1.set_xlim(-3.5e-8,3.5e-8)


ax1.yaxis.set_minor_locator(MultipleLocator(0.2))
ax2.xaxis.set_minor_locator(MultipleLocator(0.2e-8))
#ax1.xaxis.set_major_locator(MultipleLocator(10))

ax1.set_title(r'sprod: Stokes-Ekman')

ax1.axvline(0,linewidth=0.75,color='black')




cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_sprod_1c[0][:223])):
    ax2.plot(s_sprod_1c[1][:223,:][i,:],s_df['y'][:],linewidth=0.25,color=cmap(i/len(s_sprod_1c[0][:223])))

    
m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,180)
cbar1 = plt.colorbar(m1, ax=ax2, boundaries=np.linspace(0,180,len(s_S2_1c[0][:223])),ticks=[0,45,90,135,180]
                     ,pad=0.05,orientation='vertical')

ax2.set_ylim(5,0)
#ax2.set_xlim(-3.5e-8,3.5e-8)


ax2.yaxis.set_minor_locator(MultipleLocator(0.2))
ax2.xaxis.set_minor_locator(MultipleLocator(0.2e-8))
#ax2.xaxis.set_major_locator(MultipleLocator(10))

ax2.set_title(r'sprod: Stokes')

ax2.axvline(0,linewidth=0.75,color='black')


plt.tight_layout()
plt.show()

In [ ]:
fig, ((ax1,ax3),(ax2,ax4)) = plt.subplots(ncols=2,nrows=2,figsize=(8,3), sharex=True, sharey=True)



CS1 = ax1.pcolormesh(s_sprod_1c[0]/T,s_df['y'],s_sprod_1c[1].transpose(),cmap='copper_r',vmin=0,vmax=0.0025e-7)
plt.colorbar(CS1, ax=ax1)

ax1.set_ylim(5,0)

ax1.set_title('Non-Rotating',fontsize=8)
ax1.set_ylabel('Shear production',fontsize=8)


CS2 = ax2.pcolormesh(s_n_bprod_1c[0]/T,s_df['y'],s_n_bprod_1c[1].transpose(),cmap='coolwarm',vmin=-0.025e-10,vmax=0.025e-10)
plt.colorbar(CS2, ax=ax2)

ax2.set_ylim(5,0)
ax2.set_ylabel('Buoyancy production',fontsize=8)




CS3 = ax3.pcolormesh(se_sprod_1c[0]/T,se_df['y'],se_sprod_1c[1].transpose(),cmap='copper_r',vmin=0,vmax=0.0025e-7)
plt.colorbar(CS3, ax=ax3)

ax3.set_ylim(5,0)

ax3.set_title('Rotating',fontsize=8)


CS4 = ax4.pcolormesh(se_n_bprod_1c[0]/T,se_df['y'],se_n_bprod_1c[1].transpose(),cmap='coolwarm',vmin=-0.025e-10,vmax=0.025e-10)
plt.colorbar(CS4, ax=ax4)

ax4.set_ylim(5,0)

ax1.set_xticks([0,0.25,0.5,0.75,1])
ax1.set_xticklabels(['0','$\pi$/2','$\pi$','3$\pi$/2','$2\pi$'])



ax1.yaxis.set_minor_locator(MultipleLocator(0.2))
ax1.xaxis.set_minor_locator(MultipleLocator(0.05))



plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(4,1.5))

CS = plt.pcolormesh(s_diss_1c[0]/T,s_df['y'],s_diss_1c[1].transpose(),cmap='pink',vmax=0,vmin=-5e-10)
plt.colorbar(CS)

plt.ylim(5,0)

plt.show()



plt.figure(figsize=(4,1.5))

CS = plt.pcolormesh(se_diss_1c[0]/T,se_df['y'],se_diss_1c[1].transpose(),cmap='pink',vmax=0,vmin=-5e-10)
plt.colorbar(CS)

plt.ylim(5,0)

plt.show()

In [ ]:
se_df['Shear_production']

In [ ]:
se_dtmp['bprod']

In [ ]:
s_bprod_1c = cycle_average_2d(s_dtmp['bprod'], s_df['t'], s_df['y'][:], omega, 3, 6, 100)
se_bprod_1c = cycle_average_2d(se_dtmp['bprod'], se_df['t'], se_df['y'][:], omega, 3, 6, 100)

In [ ]:
s_bprod_1c[1]

In [ ]:
fig = plt.figure(figsize=(6,5))

gs = fig.add_gridspec(4,3,width_ratios=[10,10,0.5])

ax2 = fig.add_subplot(gs[0:2,0])
ax1 = fig.add_subplot(gs[0:2,1])
ax3 = fig.add_subplot(gs[2:,0])
ax4 = fig.add_subplot(gs[2:,1])

cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(se_bprod_1c[0][:223])):
    ax1.plot(se_bprod_1c[1][:223,:][i,:],se_df['y'][:],linewidth=0.25,color=cmap(i/len(se_bprod_1c[0][:223])))

    
ax1.set_ylim(5,0)
ax1.set_xlim(-3.5e-8,3.5e-8)


ax1.yaxis.set_minor_locator(MultipleLocator(0.2))
ax1.xaxis.set_minor_locator(MultipleLocator(0.2e-8))
#ax1.xaxis.set_major_locator(MultipleLocator(10))

ax1.set_title(r'Rotating')

ax1.axvline(0,linewidth=0.75,color='black')




cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_bprod_1c[0][:223])):
    ax2.plot(s_bprod_1c[1][:223,:][i,:],s_df['y'][:],linewidth=0.25,color=cmap(i/len(s_bprod_1c[0][:223])))

    
ax2.set_ylim(5,0)
ax2.set_xlim(-3.5e-8,3.5e-8)


ax2.yaxis.set_minor_locator(MultipleLocator(0.2))
ax2.xaxis.set_minor_locator(MultipleLocator(0.2e-8))
#ax2.xaxis.set_major_locator(MultipleLocator(10))

ax2.set_title(r'Non-Rotating')

ax2.axvline(0,linewidth=0.75,color='black')







cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_turner_1c[0][:223])):
    ax4.plot(se_nturn[:223,:][i,:],se_df['y'][:],linewidth=0.25,color=cmap(i/len(s_turner_1c[0][:223])))

    
ax4.set_ylim(5,0)
ax4.set_xlim(-60,0)


ax4.yaxis.set_minor_locator(MultipleLocator(0.2))
ax4.xaxis.set_minor_locator(MultipleLocator(2))
ax4.xaxis.set_major_locator(MultipleLocator(10))

#ax4.set_title(r'Turner Angle: Stokes-Ekman')

ax4.axvline(-45,linewidth=0.75,color='black')




cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_turner_1c[0][:223])):
    ax3.plot(s_nturn[:223,:][i,:],s_df['y'][:],linewidth=0.25,color=cmap(i/len(s_turner_1c[0][:223])))

    
ax3.set_ylim(5,0)
ax3.set_xlim(-60,0)


ax3.yaxis.set_minor_locator(MultipleLocator(0.2))
ax3.xaxis.set_minor_locator(MultipleLocator(2))
ax3.xaxis.set_major_locator(MultipleLocator(10))

#ax3.set_title(r'Turner Angle: Stokes')

ax3.axvline(-45,linewidth=0.75,color='black')


cax = fig.add_subplot(gs[1:3,2])

m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,180)
cbar1 = plt.colorbar(m1, cax=cax, boundaries=np.linspace(0,180,len(s_S2_1c[0][:223])),ticks=[0,45,90,135,180]
                     ,pad=0.05,orientation='vertical')


ax2.set_ylabel('Buoyancy Production',fontsize=8)
ax3.set_ylabel('Turner Angle',fontsize=8)



plt.tight_layout()
plt.show()

In [ ]:
def buoy_flux_melt(temp, sal, y, alpha, beta, Pr, Sc, nu):
    import numpy as np

    dz = np.diff(y)
    dTdz = np.diff(temp,1)/dz
    dSdz = np.diff(sal,1)/dz
    BF_melt = 9.81*(-alpha*nu/Pr*dTdz[:,0]+beta*nu/Sc*dSdz[:,0])
    
    return BF_melt

In [ ]:
s_Buo_Flux = buoy_flux_melt(s_df['Temp'], s_df['Sal'], s_df['y'], alpha, beta, Pr, Sc, nu)
se_Buo_Flux = buoy_flux_melt(se_df['Temp'], se_df['Sal'], se_df['y'], alpha, beta, Pr, Sc, nu)

In [ ]:
def cycle_average_1d(var, time, omega, start_cycle, end_cycle, dt):
    import numpy as np
    
    T = 2*np.pi/omega
    NT = int(np.floor(T/dt))
    #print(NT)
    t_cycle = np.linspace(0, T, NT)
    
    if np.floor(time[-2]/T)<end_cycle:
        print('record not', end_cycle, ' cycles long!!')
        return
    
    var_cycle = np.zeros(len(t_cycle))
    
    for ii in range(start_cycle,end_cycle):
        ttmp = time - ii*T
        tmp = np.interp(t_cycle, ttmp, var)
        var_cycle = var_cycle+tmp
        
    var_cycle = var_cycle/(end_cycle-start_cycle)
    return t_cycle, var_cycle

In [ ]:
s_Buo_Flux_1c = cycle_average_1d(s_Buo_Flux, s_df['t'], omega, 3, 6, 100)
se_Buo_Flux_1c = cycle_average_1d(se_Buo_Flux, se_df['t'], omega, 3, 6, 100)

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2,nrows=1,figsize=(10,1.5))


ax1.plot(s_Buo_Flux_1c[0]/T,s_Buo_Flux_1c[1],linewidth=0.8,color='orangered',label='Stokes')

ax1.legend(ncol=2,loc='upper right')

ax1.set_ylim(2e-11,7e-11)
ax1.set_xlim(0,1)

ax1.yaxis.set_minor_locator(MultipleLocator(0.5e-11))
ax1.xaxis.set_minor_locator(MultipleLocator(0.05))



ax2.plot(s_Buo_Flux_1c[0]/T,se_Buo_Flux_1c[1],linewidth=0.8,color='goldenrod',label='Stokes-Ekman')

ax2.legend(ncol=2,loc='upper right')

ax2.set_ylim(2e-11,7e-11)
ax2.set_xlim(0,1)

ax2.yaxis.set_minor_locator(MultipleLocator(0.5e-11))
ax2.xaxis.set_minor_locator(MultipleLocator(0.05))


plt.show()

In [ ]:
s_rho = rho0*(1 - alpha*s_df['Temp'] + beta*s_df['Sal'])
se_rho = rho0*(1 - alpha*se_df['Temp'] + beta*se_df['Sal'])

In [ ]:
s_rho_1c = cycle_average_2d(s_rho, s_df['t'], s_df['y'], omega, 3, 6, 100)
se_rho_1c = cycle_average_2d(se_rho, se_df['t'], s_df['y'], omega, 3, 6, 100)

In [ ]:
fig, (ax2, ax1) = plt.subplots(ncols=2,nrows=1,figsize=(8,3))


cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_rho_1c[0][:223])):
    ax1.plot(se_rho_1c[1][:223,:][i,:],se_df['y'][:],linewidth=0.25,color=cmap(i/len(s_rho_1c[0][:223])))

    
m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,180)
cbar1 = plt.colorbar(m1, ax=ax1, boundaries=np.linspace(0,180,len(s_S2_1c[0][:223])),ticks=[0,45,90,135,180]
                     ,pad=0.05,orientation='vertical')

ax1.set_ylim(5,0)
ax1.set_xlim(1057.806,1057.808)


ax1.yaxis.set_minor_locator(MultipleLocator(0.2))
#ax1.xaxis.set_minor_locator(MultipleLocator(2))
#ax1.xaxis.set_major_locator(MultipleLocator(10))

ax1.set_title(r'Density: Stokes-Ekman')

#ax1.axvline(-45,linewidth=0.75,color='black')




cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_rho_1c[0][:223])):
    ax2.plot(s_rho_1c[1][:223,:][i,:],s_df['y'][:],linewidth=0.25,color=cmap(i/len(s_rho_1c[0][:223])))

    
m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,180)
cbar1 = plt.colorbar(m1, ax=ax2, boundaries=np.linspace(0,180,len(s_S2_1c[0][:223])),ticks=[0,45,90,135,180]
                     ,pad=0.05,orientation='vertical')

ax2.set_ylim(5,0)
ax2.set_xlim(1057.806,1057.808)


ax2.yaxis.set_minor_locator(MultipleLocator(0.2))
#ax2.xaxis.set_minor_locator(MultipleLocator(2))
#ax2.xaxis.set_major_locator(MultipleLocator(10))

ax2.set_title(r'Density: Stokes')

#ax2.axvline(-45,linewidth=0.75,color='black')

plt.tight_layout()
plt.show()

In [ ]:
0.8*1.057e3

In [ ]:
s_dtmp

In [ ]:
s_uv_1c = cycle_average_2d(s_dtmp['uv'], s_df['t'], s_df['y'], omega, 3, 6, 100)
se_uv_1c = cycle_average_2d(se_dtmp['uv'], se_df['t'], s_df['y'], omega, 3, 6, 100)

In [ ]:
s_wv_1c = cycle_average_2d(s_dtmp['wv'], s_df['t'], s_df['y'], omega, 3, 6, 100)
se_wv_1c = cycle_average_2d(se_dtmp['wv'], se_df['t'], s_df['y'], omega, 3, 6, 100)

In [ ]:
s_uw_1c = cycle_average_2d(s_dtmp['uw'], s_df['t'], s_df['y'], omega, 3, 6, 100)
se_uw_1c = cycle_average_2d(se_dtmp['uw'], se_df['t'], s_df['y'], omega, 3, 6, 100)

In [ ]:
fig, (ax2, ax1) = plt.subplots(ncols=2,nrows=1,figsize=(8,3),sharex=True)


cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_uw_1c[0][:223])):
    ax1.plot(se_uw_1c[1][:223,:][i,:],se_df['y'][:],linewidth=0.25,color=cmap(i/len(s_uw_1c[0][:223])))

    
m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,180)
cbar1 = plt.colorbar(m1, ax=ax1, boundaries=np.linspace(0,180,len(s_S2_1c[0][:223])),ticks=[0,45,90,135,180]
                     ,pad=0.05,orientation='vertical')

ax1.set_ylim(5,0)
#ax1.set_xlim(-60,0)


ax1.yaxis.set_minor_locator(MultipleLocator(0.2))
#ax1.xaxis.set_minor_locator(MultipleLocator(2))
#ax1.xaxis.set_major_locator(MultipleLocator(10))

ax1.set_title(r'UW: Stokes-Ekman')

#ax1.axvline(-45,linewidth=0.75,color='black')




cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_uw_1c[0][:223])):
    ax2.plot(s_uw_1c[1][:223,:][i,:],s_df['y'][:],linewidth=0.25,color=cmap(i/len(s_uw_1c[0][:223])))

    
m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,180)
cbar1 = plt.colorbar(m1, ax=ax2, boundaries=np.linspace(0,180,len(s_S2_1c[0][:223])),ticks=[0,45,90,135,180]
                     ,pad=0.05,orientation='vertical')

ax2.set_ylim(5,0)
#ax2.set_xlim(-60,0)


ax2.yaxis.set_minor_locator(MultipleLocator(0.2))
#ax2.xaxis.set_minor_locator(MultipleLocator(2))
#ax2.xaxis.set_major_locator(MultipleLocator(10))

ax2.set_title(r'UW: Stokes')

#ax2.axvline(-45,linewidth=0.75,color='black')

plt.tight_layout()
plt.show()

In [ ]:
fig, (ax2, ax1) = plt.subplots(ncols=2,nrows=1,figsize=(8,3),sharex=True)


cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_uv_1c[0][:223])):
    ax1.plot(se_uv_1c[1][:223,:][i,:],se_df['y'][:],linewidth=0.25,color=cmap(i/len(s_uv_1c[0][:223])))

    
m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,180)
cbar1 = plt.colorbar(m1, ax=ax1, boundaries=np.linspace(0,180,len(s_S2_1c[0][:223])),ticks=[0,45,90,135,180]
                     ,pad=0.05,orientation='vertical')

ax1.set_ylim(5,0)
#ax1.set_xlim(-60,0)


ax1.yaxis.set_minor_locator(MultipleLocator(0.2))
#ax1.xaxis.set_minor_locator(MultipleLocator(2))
#ax1.xaxis.set_major_locator(MultipleLocator(10))

ax1.set_title(r'UV: Stokes-Ekman')

#ax1.axvline(-45,linewidth=0.75,color='black')




cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_uv_1c[0][:223])):
    ax2.plot(s_uv_1c[1][:223,:][i,:],s_df['y'][:],linewidth=0.25,color=cmap(i/len(s_uv_1c[0][:223])))

    
m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,180)
cbar1 = plt.colorbar(m1, ax=ax2, boundaries=np.linspace(0,180,len(s_S2_1c[0][:223])),ticks=[0,45,90,135,180]
                     ,pad=0.05,orientation='vertical')

ax2.set_ylim(5,0)
#ax2.set_xlim(-60,0)


ax2.yaxis.set_minor_locator(MultipleLocator(0.2))
#ax2.xaxis.set_minor_locator(MultipleLocator(2))
#ax2.xaxis.set_major_locator(MultipleLocator(10))

ax2.set_title(r'UV: Stokes')

#ax2.axvline(-45,linewidth=0.75,color='black')

plt.tight_layout()
plt.show()

In [ ]:
fig, (ax2, ax1) = plt.subplots(ncols=2,nrows=1,figsize=(8,3),sharex=True)


cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_wv_1c[0][:223])):
    ax1.plot(se_wv_1c[1][:223,:][i,:],se_df['y'][:],linewidth=0.25,color=cmap(i/len(s_wv_1c[0][:223])))

    
m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,180)
cbar1 = plt.colorbar(m1, ax=ax1, boundaries=np.linspace(0,180,len(s_S2_1c[0][:223])),ticks=[0,45,90,135,180]
                     ,pad=0.05,orientation='vertical')

ax1.set_ylim(5,0)
#ax1.set_xlim(-60,0)


ax1.yaxis.set_minor_locator(MultipleLocator(0.2))
#ax1.xaxis.set_minor_locator(MultipleLocator(2))
#ax1.xaxis.set_major_locator(MultipleLocator(10))

ax1.set_title(r'WV: Stokes-Ekman')

#ax1.axvline(-45,linewidth=0.75,color='black')




cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_wv_1c[0][:223])):
    ax2.plot(s_wv_1c[1][:223,:][i,:],s_df['y'][:],linewidth=0.25,color=cmap(i/len(s_wv_1c[0][:223])))

    
m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,180)
cbar1 = plt.colorbar(m1, ax=ax2, boundaries=np.linspace(0,180,len(s_S2_1c[0][:223])),ticks=[0,45,90,135,180]
                     ,pad=0.05,orientation='vertical')

ax2.set_ylim(5,0)
#ax2.set_xlim(-60,0)


ax2.yaxis.set_minor_locator(MultipleLocator(0.2))
#ax2.xaxis.set_minor_locator(MultipleLocator(2))
#ax2.xaxis.set_major_locator(MultipleLocator(10))

ax2.set_title(r'WV: Stokes')

#ax2.axvline(-45,linewidth=0.75,color='black')

plt.tight_layout()
plt.show()

In [ ]:
fig, ((ax0,ax01),(ax1,ax3),(ax2,ax4),(ax5,ax6)) = plt.subplots(ncols=2,nrows=4,figsize=(8,6), sharex=True, sharey=True)


CS0 = ax0.pcolormesh(s_u_star_1c[0]/T,s_df['y'],s_u_star_1c[1].transpose(),cmap='pink_r',vmin=0,vmax=0.00015)
plt.colorbar(CS0, ax=ax0)

ax0.set_ylim(5,0)

ax0.set_title('Non-Rotating',fontsize=9)
ax0.set_ylabel('u*',fontsize=9)



CS01 = ax01.pcolormesh(se_u_star_1c[0]/T,se_df['y'],se_u_star_1c[1].transpose(),cmap='pink_r',vmin=0,vmax=0.00015)
plt.colorbar(CS01, ax=ax01)

ax01.set_ylim(5,0)
ax01.set_title('Rotating',fontsize=9)





CS1 = ax1.pcolormesh(s_uv_1c[0]/T,s_df['y'],s_uv_1c[1].transpose(),cmap='PRGn',vmin=-1e-7,vmax=1e-7)
plt.colorbar(CS1, ax=ax1)

ax1.set_ylim(5,0)

#ax1.set_title('Stokes',fontsize=9)
ax1.set_ylabel('uv',fontsize=9)


CS2 = ax2.pcolormesh(s_wv_1c[0]/T,s_df['y'],s_wv_1c[1].transpose(),cmap='coolwarm',vmin=-1e-7,vmax=1e-7)
plt.colorbar(CS2, ax=ax2)

ax2.set_ylim(5,0)
ax2.set_ylabel('wv',fontsize=9)




CS3 = ax3.pcolormesh(se_uv_1c[0]/T,se_df['y'],se_uv_1c[1].transpose(),cmap='PRGn',vmin=-1e-7,vmax=1e-7)
plt.colorbar(CS3, ax=ax3)

ax3.set_ylim(5,0)

#ax3.set_title('Stokes-Ekman',fontsize=9)


CS4 = ax4.pcolormesh(se_wv_1c[0]/T,se_df['y'],se_wv_1c[1].transpose(),cmap='coolwarm',vmin=-1e-7,vmax=1e-7)
plt.colorbar(CS4, ax=ax4)

ax4.set_ylim(5,0)




CS5 = ax5.pcolormesh(s_uw_1c[0]/T,s_df['y'],s_uw_1c[1].transpose(),cmap='PuOr_r',vmin=-1e-7,vmax=1e-7)
plt.colorbar(CS5, ax=ax5)

ax5.set_ylim(5,0)



CS6 = ax6.pcolormesh(se_uw_1c[0]/T,se_df['y'],se_uw_1c[1].transpose(),cmap='PuOr_r',vmin=-1e-7,vmax=1e-7)
plt.colorbar(CS6, ax=ax6)

ax6.set_ylim(5,0)
ax5.set_ylabel('uw',fontsize=9)



ax1.yaxis.set_minor_locator(MultipleLocator(0.2))
ax1.xaxis.set_minor_locator(MultipleLocator(0.05))

ax2.yaxis.set_minor_locator(MultipleLocator(0.2))
ax2.xaxis.set_minor_locator(MultipleLocator(0.05))

ax3.yaxis.set_minor_locator(MultipleLocator(0.2))
ax3.xaxis.set_minor_locator(MultipleLocator(0.05))

ax4.yaxis.set_minor_locator(MultipleLocator(0.2))
ax4.xaxis.set_minor_locator(MultipleLocator(0.05))

ax5.yaxis.set_minor_locator(MultipleLocator(0.2))
ax5.xaxis.set_minor_locator(MultipleLocator(0.05))

ax6.yaxis.set_minor_locator(MultipleLocator(0.2))
ax6.xaxis.set_minor_locator(MultipleLocator(0.05))

ax0.yaxis.set_minor_locator(MultipleLocator(0.2))
ax0.xaxis.set_minor_locator(MultipleLocator(0.05))

ax01.yaxis.set_minor_locator(MultipleLocator(0.2))
ax01.xaxis.set_minor_locator(MultipleLocator(0.05))








plt.tight_layout()
plt.show()

In [ ]:
s_Temp_1c = cycle_average_2d(s_df['Temp'], s_df['t'], s_df['y'], omega, 3, 6, 100)
se_Temp_1c = cycle_average_2d(se_df['Temp'], se_df['t'], s_df['y'], omega, 3, 6, 100)

In [ ]:
fig, (ax2, ax1) = plt.subplots(ncols=2,nrows=1,figsize=(8,3))


cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_Temp_1c[0][:223])):
    ax1.plot(se_Temp_1c[1][:223,:][i,:],se_df['y'][:],linewidth=0.25,color=cmap(i/len(s_Temp_1c[0][:223])))

    
m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,180)
cbar1 = plt.colorbar(m1, ax=ax1, boundaries=np.linspace(0,180,len(s_S2_1c[0][:223])),ticks=[0,45,90,135,180]
                     ,pad=0.05,orientation='vertical')

ax1.set_ylim(5,0)
#ax1.set_xlim(-60,0)


ax1.yaxis.set_minor_locator(MultipleLocator(0.2))
#ax1.xaxis.set_minor_locator(MultipleLocator(2))
#ax1.xaxis.set_major_locator(MultipleLocator(10))

ax1.set_title(r'Temperature: Stokes-Ekman')

#ax1.axvline(-45,linewidth=0.75,color='black')




cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_Temp_1c[0][:223])):
    ax2.plot(s_Temp_1c[1][:223,:][i,:],s_df['y'][:],linewidth=0.25,color=cmap(i/len(s_Temp_1c[0][:223])))

    
m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,180)
cbar1 = plt.colorbar(m1, ax=ax2, boundaries=np.linspace(0,180,len(s_S2_1c[0][:223])),ticks=[0,45,90,135,180]
                     ,pad=0.05,orientation='vertical')

ax2.set_ylim(5,0)
#ax2.set_xlim(-60,0)


ax2.yaxis.set_minor_locator(MultipleLocator(0.2))
#ax2.xaxis.set_minor_locator(MultipleLocator(2))
#ax2.xaxis.set_major_locator(MultipleLocator(10))

ax2.set_title(r'Temperature: Stokes')

#ax2.axvline(-45,linewidth=0.75,color='black')

plt.tight_layout()
plt.show()

In [ ]:
s_Sal_1c = cycle_average_2d(s_df['Sal'], s_df['t'], s_df['y'], omega, 3, 6, 100)
se_Sal_1c = cycle_average_2d(se_df['Sal'], se_df['t'], s_df['y'], omega, 3, 6, 100)

In [ ]:
fig, (ax2, ax1) = plt.subplots(ncols=2,nrows=1,figsize=(8,3))


cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_Sal_1c[0][:223])):
    ax1.plot(se_Sal_1c[1][:223,:][i,:],se_df['y'][:],linewidth=0.25,color=cmap(i/len(s_Sal_1c[0][:223])))

    
m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,180)
cbar1 = plt.colorbar(m1, ax=ax1, boundaries=np.linspace(0,180,len(s_S2_1c[0][:223])),ticks=[0,45,90,135,180]
                     ,pad=0.05,orientation='vertical')

ax1.set_ylim(5,0)
#ax1.set_xlim(-60,0)


ax1.yaxis.set_minor_locator(MultipleLocator(0.2))
#ax1.xaxis.set_minor_locator(MultipleLocator(2))
#ax1.xaxis.set_major_locator(MultipleLocator(10))

ax1.set_title(r'Salinity: Stokes-Ekman')

#ax1.axvline(-45,linewidth=0.75,color='black')




cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_Sal_1c[0][:223])):
    ax2.plot(s_Sal_1c[1][:223,:][i,:],s_df['y'][:],linewidth=0.25,color=cmap(i/len(s_Sal_1c[0][:223])))

    
m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,180)
cbar1 = plt.colorbar(m1, ax=ax2, boundaries=np.linspace(0,180,len(s_S2_1c[0][:223])),ticks=[0,45,90,135,180]
                     ,pad=0.05,orientation='vertical')

ax2.set_ylim(5,0)
#ax2.set_xlim(-60,0)


ax2.yaxis.set_minor_locator(MultipleLocator(0.2))
#ax2.xaxis.set_minor_locator(MultipleLocator(2))
#ax2.xaxis.set_major_locator(MultipleLocator(10))

ax2.set_title(r'Salinity: Stokes')

#ax2.axvline(-45,linewidth=0.75,color='black')

plt.tight_layout()
plt.show()

In [ ]:
s_Ri_1c_gf = sci.ndimage.gaussian_filter1d(s_Ri_1c[1],sigma=2,axis=1)
se_Ri_1c_gf = sci.ndimage.gaussian_filter1d(se_Ri_1c[1],sigma=2,axis=1)

In [ ]:
fig = plt.figure(figsize=(8,5))

gs = fig.add_gridspec(4,6,width_ratios=[10,10,10,10,10,0.75])

ax1 = fig.add_subplot(gs[2:,0])
ax2 = fig.add_subplot(gs[2:,1], sharey=ax1)
ax3 = fig.add_subplot(gs[2:,2], sharey=ax1)
ax4 = fig.add_subplot(gs[2:,3], sharey=ax1)
ax5 = fig.add_subplot(gs[2:,4], sharey=ax1)

ax1_1 = fig.add_subplot(gs[:2,0], sharex=ax1)
ax2_1 = fig.add_subplot(gs[:2,1], sharex=ax2, sharey=ax1_1)
ax3_1 = fig.add_subplot(gs[:2,2], sharex=ax3, sharey=ax1_1)
ax4_1 = fig.add_subplot(gs[:2,3], sharex=ax4, sharey=ax1_1)
ax5_1 = fig.add_subplot(gs[:2,4], sharex=ax5, sharey=ax1_1)

cax = fig.add_subplot(gs[1:3,5])

plt.setp(ax2.get_yticklabels(), visible=False)
plt.setp(ax3.get_yticklabels(), visible=False)
plt.setp(ax4.get_yticklabels(), visible=False)
plt.setp(ax5.get_yticklabels(), visible=False)

plt.setp(ax2_1.get_yticklabels(), visible=False)
plt.setp(ax3_1.get_yticklabels(), visible=False)
plt.setp(ax4_1.get_yticklabels(), visible=False)
plt.setp(ax5_1.get_yticklabels(), visible=False)

plt.setp(ax1_1.get_xticklabels(), visible=False)
plt.setp(ax2_1.get_xticklabels(), visible=False)
plt.setp(ax3_1.get_xticklabels(), visible=False)
plt.setp(ax4_1.get_xticklabels(), visible=False)
plt.setp(ax5_1.get_xticklabels(), visible=False)


cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_N2_1c[0][:223])):
    ax1.plot(se_N2_1c[1][:223,:][i,:],se_df['y'],linewidth=0.25,color=cmap(i/len(s_N2_1c[0][:223])))

ax1.set_ylim(5,0)
ax1.set_xlim(-0.1e-5,1.8e-5)

ax1.yaxis.set_minor_locator(MultipleLocator(0.2))
#ax1.xaxis.set_minor_locator(MultipleLocator(0.1e-5))

#ax1.set_title(r'N$^{2}$: Stokes-Ekman')




cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_N2_1c[0][:223])):
    ax1_1.plot(s_N2_1c[1][:223,:][i,:],s_df['y'],linewidth=0.25,color=cmap(i/len(s_N2_1c[0][:223])))

ax1_1.set_ylim(5,0)
ax1_1.set_xlim(-0.1e-5,1.8e-5)

ax1_1.yaxis.set_minor_locator(MultipleLocator(0.2))
#ax1_1.xaxis.set_minor_locator(MultipleLocator(0.1e-5))

ax1_1.set_title(r'N$^{2}$')




cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(se_u_star_1c[0][:223])):
    ax2.plot(se_u_star_1c[1][:223,:][i,:],se_df['y'],linewidth=0.25,color=cmap(i/len(se_u_star_1c[0][:223])))

ax2.set_ylim(5,0)
ax2.set_xlim(0,1.2e-3)

ax2.yaxis.set_minor_locator(MultipleLocator(0.2))
#ax2.xaxis.set_minor_locator(MultipleLocator(0.1e-5))

#ax2.set_title(r'u*: Stokes-Ekman')





cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_u_star_1c[0][:223])):
    ax2_1.plot(s_u_star_1c[1][:223,:][i,:],s_df['y'],linewidth=0.25,color=cmap(i/len(s_u_star_1c[0][:223])))

ax2_1.set_ylim(5,0)
ax2_1.set_xlim(0,1.2e-3)

ax2_1.yaxis.set_minor_locator(MultipleLocator(0.2))
#ax2_1.xaxis.set_minor_locator(MultipleLocator(0.1e-5))

ax2_1.set_title(r'u*')





cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_TKE_1c[0][:223])):
    ax3.plot(se_TKE_1c[1][:223,:][i,:],se_df['y'],linewidth=0.25,color=cmap(i/len(s_TKE_1c[0][:223])))

ax3.set_ylim(5,0)
ax3.set_xlim(0,1.25e-5)

ax3.yaxis.set_minor_locator(MultipleLocator(0.2))
#ax3.xaxis.set_minor_locator(MultipleLocator(0.2e-5))

#ax3.set_title(r'TKE: Stokes-Ekman')




cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_TKE_1c[0][:223])):
    ax3_1.plot(s_TKE_1c[1][:223,:][i,:],s_df['y'],linewidth=0.25,color=cmap(i/len(s_TKE_1c[0][:223])))

ax3_1.set_ylim(5,0)
ax3_1.set_xlim(0,1.25e-5)

ax3_1.yaxis.set_minor_locator(MultipleLocator(0.2))
#ax3_1.xaxis.set_minor_locator(MultipleLocator(0.2e-5))

ax3_1.set_title(r'TKE')





cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_S2_1c[0][:223])):
    ax4.plot(se_S2_1c[1][:223,:][i,:],se_df['y'],linewidth=0.25,color=cmap(i/len(s_S2_1c[0][:223])))

ax4.set_ylim(5,0)
ax4.set_xlim(0,6e-5)

ax4.yaxis.set_minor_locator(MultipleLocator(0.2))
#ax4.xaxis.set_minor_locator(MultipleLocator(0.2e-5))

#ax4.set_title(r'S$^{2}$: Stokes-Ekman')





cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_S2_1c[0][:223])):
    ax4_1.plot(s_S2_1c[1][:223,:][i,:],s_df['y'],linewidth=0.25,color=cmap(i/len(s_S2_1c[0][:223])))
    
ax4_1.set_ylim(5,0)
ax4_1.set_xlim(0,6e-5)

ax4_1.yaxis.set_minor_locator(MultipleLocator(0.2))
#ax4_1.xaxis.set_minor_locator(MultipleLocator(0.2e-5))

ax4_1.set_title(r'S$^{2}$')






cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_Ri_1c[0][:223])):
    ax5.plot(se_Ri_1c_gf[:223,:][i,:],se_df['y'],linewidth=0.25,color=cmap(i/len(s_Ri_1c[0][:223])))

ax5.set_ylim(5,0)
ax5.set_xlim(0,100000)

ax5.yaxis.set_minor_locator(MultipleLocator(0.2))
#ax5.xaxis.set_minor_locator(MultipleLocator(0.1))

#ax5.set_title(r'R$_{g}$: Stokes-Ekman')





cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_Ri_1c[0][:223])):
    ax5_1.plot(s_Ri_1c_gf[:223,:][i,:],s_df['y'],linewidth=0.25,color=cmap(i/len(s_Ri_1c[0][:223])))

ax5_1.set_ylim(5,0)
ax5_1.set_xlim(0,100000)

ax5_1.yaxis.set_minor_locator(MultipleLocator(0.2))
#ax5_1.xaxis.set_minor_locator(MultipleLocator(0.1))

ax5_1.set_title(r'R$_{g}$')




m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,180)
cbar1 = plt.colorbar(m1, cax=cax, boundaries=np.linspace(0,180,len(s_S2_1c[0][:223])),ticks=[0,45,90,135,180]
                     ,pad=0.05,orientation='vertical')


ax1_1.set_ylabel('Stokes')
ax1.set_ylabel('Stokes-Ekman')




plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(8,5))

gs = fig.add_gridspec(4,6,width_ratios=[10,10,10,10,10,0.75])

ax1 = fig.add_subplot(gs[2:,0])
ax2 = fig.add_subplot(gs[2:,1])
ax3 = fig.add_subplot(gs[2:,2])
ax4 = fig.add_subplot(gs[2:,3])
ax5 = fig.add_subplot(gs[2:,4])

ax1_1 = fig.add_subplot(gs[:2,0])
ax2_1 = fig.add_subplot(gs[:2,1])
ax3_1 = fig.add_subplot(gs[:2,2])
ax4_1 = fig.add_subplot(gs[:2,3])
ax5_1 = fig.add_subplot(gs[:2,4])

cax = fig.add_subplot(gs[1:3,5])



cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_N2_1c[0][:223])):
    ax1.plot(se_N2_1c[1][:223,:][i,:],se_df['y'],linewidth=0.25,color=cmap(i/len(s_N2_1c[0][:223])))

ax1.set_ylim(5,0)
ax1.set_xlim(-0.1e-5,1.8e-5)

ax1.yaxis.set_minor_locator(MultipleLocator(0.2))
ax1.xaxis.set_minor_locator(MultipleLocator(0.1e-5))

#ax1.set_title(r'N$^{2}$: Stokes-Ekman')




cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_N2_1c[0][:223])):
    ax1_1.plot(s_N2_1c[1][:223,:][i,:],s_df['y'],linewidth=0.25,color=cmap(i/len(s_N2_1c[0][:223])))

ax1_1.set_ylim(5,0)
ax1_1.set_xlim(-0.1e-5,1.8e-5)

ax1_1.yaxis.set_minor_locator(MultipleLocator(0.2))
ax1_1.xaxis.set_minor_locator(MultipleLocator(0.1e-5))

ax1_1.set_title(r'N$^{2}$')




cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(se_u_star_1c[0][:223])):
    ax2.plot(se_u_star_1c[1][:223,:][i,:],se_df['y'],linewidth=0.25,color=cmap(i/len(se_u_star_1c[0][:223])))

ax2.set_ylim(5,0)
ax2.set_xlim(0,1.2e-3)

ax2.yaxis.set_minor_locator(MultipleLocator(0.2))
ax2.xaxis.set_minor_locator(MultipleLocator(0.1e-3))

#ax2.set_title(r'u*: Stokes-Ekman')





cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_u_star_1c[0][:223])):
    ax2_1.plot(s_u_star_1c[1][:223,:][i,:],s_df['y'],linewidth=0.25,color=cmap(i/len(s_u_star_1c[0][:223])))

ax2_1.set_ylim(5,0)
ax2_1.set_xlim(0,1.2e-3)

ax2_1.yaxis.set_minor_locator(MultipleLocator(0.2))
ax2_1.xaxis.set_minor_locator(MultipleLocator(0.1e-3))

ax2_1.set_title(r'u*')





cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_TKE_1c[0][:223])):
    ax3.plot(se_TKE_1c[1][:223,:][i,:],se_df['y'],linewidth=0.25,color=cmap(i/len(s_TKE_1c[0][:223])))

ax3.set_ylim(5,0)
ax3.set_xlim(0,1.25e-5)

ax3.yaxis.set_minor_locator(MultipleLocator(0.2))
ax3.xaxis.set_minor_locator(MultipleLocator(0.1e-5))

#ax3.set_title(r'TKE: Stokes-Ekman')




cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_TKE_1c[0][:223])):
    ax3_1.plot(s_TKE_1c[1][:223,:][i,:],s_df['y'],linewidth=0.25,color=cmap(i/len(s_TKE_1c[0][:223])))

ax3_1.set_ylim(5,0)
ax3_1.set_xlim(0,1.25e-5)

ax3_1.yaxis.set_minor_locator(MultipleLocator(0.2))
ax3_1.xaxis.set_minor_locator(MultipleLocator(0.1e-5))

ax3_1.set_title(r'TKE')





cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_S2_1c[0][:223])):
    ax4.plot(se_S2_1c[1][:223,:][i,:],se_df['y'],linewidth=0.25,color=cmap(i/len(s_S2_1c[0][:223])))

ax4.set_ylim(5,0)
ax4.set_xlim(0,6e-5)

ax4.yaxis.set_minor_locator(MultipleLocator(0.2))
ax4.xaxis.set_minor_locator(MultipleLocator(0.5e-5))

#ax4.set_title(r'S$^{2}$: Stokes-Ekman')





cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_S2_1c[0][:223])):
    ax4_1.plot(s_S2_1c[1][:223,:][i,:],s_df['y'],linewidth=0.25,color=cmap(i/len(s_S2_1c[0][:223])))
    
ax4_1.set_ylim(5,0)
ax4_1.set_xlim(0,6e-5)

ax4_1.yaxis.set_minor_locator(MultipleLocator(0.2))
ax4_1.xaxis.set_minor_locator(MultipleLocator(0.5e-5))

ax4_1.set_title(r'S$^{2}$')






cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_Ri_1c[0][:223])):
    ax5.plot(se_Ri_1c_gf[:223,:][i,:],se_df['y'],linewidth=0.25,color=cmap(i/len(s_Ri_1c[0][:223])))

ax5.set_ylim(5,0)
ax5.set_xlim(0,0.5)

ax5.yaxis.set_minor_locator(MultipleLocator(0.2))
ax5.xaxis.set_minor_locator(MultipleLocator(0.05))

#ax5.set_title(r'R$_{g}$: Stokes-Ekman')





cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_Ri_1c[0][:223])):
    ax5_1.plot(s_Ri_1c_gf[:223,:][i,:],s_df['y'],linewidth=0.25,color=cmap(i/len(s_Ri_1c[0][:223])))

ax5_1.set_ylim(5,0)
ax5_1.set_xlim(0,0.5)

ax5_1.yaxis.set_minor_locator(MultipleLocator(0.2))
ax5_1.xaxis.set_minor_locator(MultipleLocator(0.05))

ax5_1.set_title(r'R$_{g}$')




m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,180)
cbar1 = plt.colorbar(m1, cax=cax, boundaries=np.linspace(0,180,len(s_S2_1c[0][:223])),ticks=[0,45,90,135,180]
                     ,pad=0.05,orientation='vertical')
cbar1.ax.set_yticklabels(['0','$\pi$/4','$\pi$/2','3$\pi$/4','$\pi$'])

ax1_1.set_ylabel('Stokes')
ax1.set_ylabel('Stokes-Ekman')



ax1.set_xticks([0,0.5e-5,1e-5,1.5e-5])
ax2.set_xticks([0,0.5e-3,1e-3])
ax3.set_xticks([0,0.5e-5,1e-5])
ax4.set_xticks([0,2e-5,4e-5,6e-5])
ax5.set_xticks([0,0.2,0.4])

ax1_1.set_xticks([0,0.5e-5,1e-5,1.5e-5])
ax2_1.set_xticks([0,0.5e-3,1e-3])
ax3_1.set_xticks([0,0.5e-5,1e-5])
ax4_1.set_xticks([0,2e-5,4e-5,6e-5])
ax5_1.set_xticks([0,0.2,0.4])

ax1_1.set_xticklabels([])
ax2_1.set_xticklabels([])
ax3_1.set_xticklabels([])
ax4_1.set_xticklabels([])
ax5_1.set_xticklabels([])




plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(6.5,5))

gs = fig.add_gridspec(4,5,width_ratios=[10,10,10,10,0.75])

ax1 = fig.add_subplot(gs[2:,0])
ax3 = fig.add_subplot(gs[2:,1])
ax4 = fig.add_subplot(gs[2:,2])
ax5 = fig.add_subplot(gs[2:,3])

ax1_1 = fig.add_subplot(gs[:2,0])
ax3_1 = fig.add_subplot(gs[:2,1])
ax4_1 = fig.add_subplot(gs[:2,2])
ax5_1 = fig.add_subplot(gs[:2,3])

cax = fig.add_subplot(gs[1:3,4])



cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_N2_1c[0][:223])):
    ax1.plot(se_N2_1c[1][:223,:][i,:],se_df['y'],linewidth=0.25,color=cmap(i/len(s_N2_1c[0][:223])))

ax1.set_ylim(5,0)
ax1.set_xlim(-0.1e-5,1.8e-5)

ax1.yaxis.set_minor_locator(MultipleLocator(0.2))
ax1.xaxis.set_minor_locator(MultipleLocator(0.1e-5))

#ax1.set_title(r'N$^{2}$: Stokes-Ekman')




cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_N2_1c[0][:223])):
    ax1_1.plot(s_N2_1c[1][:223,:][i,:],s_df['y'],linewidth=0.25,color=cmap(i/len(s_N2_1c[0][:223])))

ax1_1.set_ylim(5,0)
ax1_1.set_xlim(-0.1e-5,1.8e-5)

ax1_1.yaxis.set_minor_locator(MultipleLocator(0.2))
ax1_1.xaxis.set_minor_locator(MultipleLocator(0.1e-5))

ax1_1.set_title(r'N$^{2}$')







cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_TKE_1c[0][:223])):
    ax3.plot(se_TKE_1c[1][:223,:][i,:],se_df['y'],linewidth=0.25,color=cmap(i/len(s_TKE_1c[0][:223])))

ax3.set_ylim(5,0)
ax3.set_xlim(0,1.25e-5)

ax3.yaxis.set_minor_locator(MultipleLocator(0.2))
ax3.xaxis.set_minor_locator(MultipleLocator(0.1e-5))

#ax3.set_title(r'TKE: Stokes-Ekman')




cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_TKE_1c[0][:223])):
    ax3_1.plot(s_TKE_1c[1][:223,:][i,:],s_df['y'],linewidth=0.25,color=cmap(i/len(s_TKE_1c[0][:223])))

ax3_1.set_ylim(5,0)
ax3_1.set_xlim(0,1.25e-5)

ax3_1.yaxis.set_minor_locator(MultipleLocator(0.2))
ax3_1.xaxis.set_minor_locator(MultipleLocator(0.1e-5))

ax3_1.set_title(r'TKE')





cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_S2_1c[0][:223])):
    ax4.plot(se_S2_1c[1][:223,:][i,:],se_df['y'],linewidth=0.25,color=cmap(i/len(s_S2_1c[0][:223])))

ax4.set_ylim(5,0)
ax4.set_xlim(0,6e-5)

ax4.yaxis.set_minor_locator(MultipleLocator(0.2))
ax4.xaxis.set_minor_locator(MultipleLocator(0.5e-5))

#ax4.set_title(r'S$^{2}$: Stokes-Ekman')





cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_S2_1c[0][:223])):
    ax4_1.plot(s_S2_1c[1][:223,:][i,:],s_df['y'],linewidth=0.25,color=cmap(i/len(s_S2_1c[0][:223])))
    
ax4_1.set_ylim(5,0)
ax4_1.set_xlim(0,6e-5)

ax4_1.yaxis.set_minor_locator(MultipleLocator(0.2))
ax4_1.xaxis.set_minor_locator(MultipleLocator(0.5e-5))

ax4_1.set_title(r'S$^{2}$')






cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_Ri_1c[0][:223])):
    ax5.plot(se_Ri_1c_gf[:223,:][i,:],se_df['y'],linewidth=0.25,color=cmap(i/len(s_Ri_1c[0][:223])))

ax5.set_ylim(5,0)
ax5.set_xlim(0,0.5)

ax5.yaxis.set_minor_locator(MultipleLocator(0.2))
ax5.xaxis.set_minor_locator(MultipleLocator(0.05))

#ax5.set_title(r'R$_{g}$: Stokes-Ekman')





cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_Ri_1c[0][:223])):
    ax5_1.plot(s_Ri_1c_gf[:223,:][i,:],s_df['y'],linewidth=0.25,color=cmap(i/len(s_Ri_1c[0][:223])))

ax5_1.set_ylim(5,0)
ax5_1.set_xlim(0,0.5)

ax5_1.yaxis.set_minor_locator(MultipleLocator(0.2))
ax5_1.xaxis.set_minor_locator(MultipleLocator(0.05))

ax5_1.set_title(r'R$_{g}$')




m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,180)
cbar1 = plt.colorbar(m1, cax=cax, boundaries=np.linspace(0,180,len(s_S2_1c[0][:223])),ticks=[0,45,90,135,180]
                     ,pad=0.05,orientation='vertical')
cbar1.ax.set_yticklabels(['0','$\pi$/4','$\pi$/2','3$\pi$/4','$\pi$'])

ax1_1.set_ylabel('Non-Rotating')
ax1.set_ylabel('Rotating')



ax1.set_xticks([0,0.5e-5,1e-5,1.5e-5])
ax2.set_xticks([0,0.5e-3,1e-3])
ax3.set_xticks([0,0.5e-5,1e-5])
ax4.set_xticks([0,2e-5,4e-5,6e-5])
ax5.set_xticks([0,0.2,0.4])

ax1_1.set_xticks([0,0.5e-5,1e-5,1.5e-5])
ax2_1.set_xticks([0,0.5e-3,1e-3])
ax3_1.set_xticks([0,0.5e-5,1e-5])
ax4_1.set_xticks([0,2e-5,4e-5,6e-5])
ax5_1.set_xticks([0,0.2,0.4])

ax1_1.set_xticklabels([])
ax2_1.set_xticklabels([])
ax3_1.set_xticklabels([])
ax4_1.set_xticklabels([])
ax5_1.set_xticklabels([])




plt.tight_layout()
plt.show()

In [ ]:
1+1

In [ ]:
fig = plt.figure(figsize=(6.5,5))

gs = fig.add_gridspec(4,5,width_ratios=[10,10,10,10,0.75])

ax1 = fig.add_subplot(gs[2:,0])
ax3 = fig.add_subplot(gs[2:,1])
ax4 = fig.add_subplot(gs[2:,2])
ax5 = fig.add_subplot(gs[2:,3])

ax1_1 = fig.add_subplot(gs[:2,0])
ax3_1 = fig.add_subplot(gs[:2,1])
ax4_1 = fig.add_subplot(gs[:2,2])
ax5_1 = fig.add_subplot(gs[:2,3])

cax = fig.add_subplot(gs[1:3,4])



cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_u_star_1c[0][:223])):
    ax1.plot(se_u_star_1c[1][:223,:][i,:],se_df['y'],linewidth=0.25,color=cmap(i/len(s_u_star_1c[0][:223])))

ax1.set_ylim(5,0)
ax1.set_xlim(0,1.2e-3)

ax1.yaxis.set_minor_locator(MultipleLocator(0.2))
ax1.xaxis.set_minor_locator(MultipleLocator(0.1e-3))

#ax1.set_title(r'N$^{2}$: Stokes-Ekman')




cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_u_star_1c[0][:223])):
    ax1_1.plot(s_u_star_1c[1][:223,:][i,:],s_df['y'],linewidth=0.25,color=cmap(i/len(s_u_star_1c[0][:223])))

ax1_1.set_ylim(5,0)
ax1_1.set_xlim(0,1.2e-3)

ax1_1.yaxis.set_minor_locator(MultipleLocator(0.2))
ax1_1.xaxis.set_minor_locator(MultipleLocator(0.1e-3))

ax1_1.set_title(r'u*')







cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_uv_1c[0][:223])):
    ax3.plot(se_uv_1c[1][:223,:][i,:],se_df['y'],linewidth=0.25,color=cmap(i/len(s_uv_1c[0][:223])))

ax3.set_ylim(5,0)
ax3.set_xlim(-0.4e-6,1.4e-6)

ax3.yaxis.set_minor_locator(MultipleLocator(0.2))
ax3.xaxis.set_minor_locator(MultipleLocator(0.1e-6))

#ax3.set_title(r'TKE: Stokes-Ekman')




cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_uv_1c[0][:223])):
    ax3_1.plot(s_uv_1c[1][:223,:][i,:],s_df['y'],linewidth=0.25,color=cmap(i/len(s_uv_1c[0][:223])))

ax3_1.set_ylim(5,0)
ax3_1.set_xlim(-0.4e-6,1.4e-6)

ax3_1.yaxis.set_minor_locator(MultipleLocator(0.2))
ax3_1.xaxis.set_minor_locator(MultipleLocator(0.1e-6))

ax3_1.set_title(r'uv')





cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_uw_1c[0][:223])):
    ax4.plot(se_uw_1c[1][:223,:][i,:],se_df['y'],linewidth=0.25,color=cmap(i/len(s_uw_1c[0][:223])))

ax4.set_ylim(5,0)
ax4.set_xlim(-1.75e-6,1.75e-6)

ax4.yaxis.set_minor_locator(MultipleLocator(0.2))
ax4.xaxis.set_minor_locator(MultipleLocator(0.25e-6))

#ax4.set_title(r'S$^{2}$: Stokes-Ekman')





cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_uw_1c[0][:223])):
    ax4_1.plot(s_uw_1c[1][:223,:][i,:],s_df['y'],linewidth=0.25,color=cmap(i/len(s_uw_1c[0][:223])))
    
ax4_1.set_ylim(5,0)
ax4_1.set_xlim(-1.75e-6,1.75e-6)

ax4_1.yaxis.set_minor_locator(MultipleLocator(0.2))
ax4_1.xaxis.set_minor_locator(MultipleLocator(0.25e-6))

ax4_1.set_title(r'uw')






cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_wv_1c[0][:223])):
    ax5.plot(se_wv_1c[1][:223,:][i,:],se_df['y'],linewidth=0.25,color=cmap(i/len(s_wv_1c[0][:223])))

ax5.set_ylim(5,0)
ax5.set_xlim(-3.25e-7,3.25e-7)

ax5.yaxis.set_minor_locator(MultipleLocator(0.2))
ax5.xaxis.set_minor_locator(MultipleLocator(0.5e-7))

#ax5.set_title(r'R$_{g}$: Stokes-Ekman')





cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_wv_1c[0][:223])):
    ax5_1.plot(s_wv_1c[1][:223,:][i,:],s_df['y'],linewidth=0.25,color=cmap(i/len(s_wv_1c[0][:223])))

ax5_1.set_ylim(5,0)
ax5_1.set_xlim(-3.25e-7,3.25e-7)

ax5_1.yaxis.set_minor_locator(MultipleLocator(0.2))
ax5_1.xaxis.set_minor_locator(MultipleLocator(0.5e-7))

ax5_1.set_title(r'wv')




m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,180)
cbar1 = plt.colorbar(m1, cax=cax, boundaries=np.linspace(0,180,len(s_S2_1c[0][:223])),ticks=[0,45,90,135,180]
                     ,pad=0.05,orientation='vertical')
cbar1.ax.set_yticklabels(['0','$\pi$/4','$\pi$/2','3$\pi$/4','$\pi$'])

ax1_1.set_ylabel('Non-Rotating')
ax1.set_ylabel('Rotating')



ax1.set_xticks([0,0.0005,0.001])
ax3.set_xticks([0,0.5e-6,1e-6])
ax4.set_xticks([-1e-6,0,1e-6])
ax5.set_xticks([-2.5e-7,0,2.5e-7])

ax1_1.set_xticks([0,0.0005,0.001])
ax3_1.set_xticks([0,0.5e-6,1e-6])
ax4_1.set_xticks([-1e-6,0,1e-6])
ax5_1.set_xticks([-2.5e-7,0,2.5e-7])

ax1_1.set_xticklabels([])
ax3_1.set_xticklabels([])
ax4_1.set_xticklabels([])
ax5_1.set_xticklabels([])




plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(5.5,5))

gs = fig.add_gridspec(4,4,width_ratios=[10,10,10,0.75])

ax1 = fig.add_subplot(gs[2:,0])
ax3 = fig.add_subplot(gs[2:,1])
ax4 = fig.add_subplot(gs[2:,2])

ax1_1 = fig.add_subplot(gs[:2,0])
ax3_1 = fig.add_subplot(gs[:2,1])
ax4_1 = fig.add_subplot(gs[:2,2])

cax = fig.add_subplot(gs[1:3,3])



cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_rho_1c[0][:223])):
    ax1.plot(se_rho_1c[1][:223,:][i,:],se_df['y'],linewidth=0.25,color=cmap(i/len(s_rho_1c[0][:223])))

ax1.set_ylim(5,0)
ax1.set_xlim(1057.806,1057.808)

ax1.yaxis.set_minor_locator(MultipleLocator(0.2))
#ax1.xaxis.set_minor_locator(MultipleLocator(0.1e-3))

#ax1.set_title(r'N$^{2}$: Stokes-Ekman')




cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_rho_1c[0][:223])):
    ax1_1.plot(s_rho_1c[1][:223,:][i,:],s_df['y'],linewidth=0.25,color=cmap(i/len(s_rho_1c[0][:223])))

ax1_1.set_ylim(5,0)
ax1_1.set_xlim(1057.806,1057.808)

ax1_1.yaxis.set_minor_locator(MultipleLocator(0.2))
#ax1_1.xaxis.set_minor_locator(MultipleLocator(0.1e-3))

ax1_1.set_title(r'Density')







cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_Temp_1c[0][:223])):
    ax3.plot(se_Temp_1c[1][:223,:][i,:],se_df['y'],linewidth=0.25,color=cmap(i/len(s_Temp_1c[0][:223])))

ax3.set_ylim(5,0)
ax3.set_xlim(-2.2761,-2.2749)

ax3.yaxis.set_minor_locator(MultipleLocator(0.2))
#ax3.xaxis.set_minor_locator(MultipleLocator(0.1e-6))

#ax3.set_title(r'TKE: Stokes-Ekman')




cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_Temp_1c[0][:223])):
    ax3_1.plot(s_Temp_1c[1][:223,:][i,:],s_df['y'],linewidth=0.25,color=cmap(i/len(s_Temp_1c[0][:223])))

ax3_1.set_ylim(5,0)
ax3_1.set_xlim(-2.2761,-2.2749)

ax3_1.yaxis.set_minor_locator(MultipleLocator(0.2))
#ax3_1.xaxis.set_minor_locator(MultipleLocator(0.1e-6))

ax3_1.set_title(r'Temperature')





cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_Sal_1c[0][:223])):
    ax4.plot(se_Sal_1c[1][:223,:][i,:],se_df['y'],linewidth=0.25,color=cmap(i/len(s_Sal_1c[0][:223])))

ax4.set_ylim(5,0)
ax4.set_xlim(34.4995,34.502)

ax4.yaxis.set_minor_locator(MultipleLocator(0.2))
#ax4.xaxis.set_minor_locator(MultipleLocator(0.25e-6))

#ax4.set_title(r'S$^{2}$: Stokes-Ekman')





cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_Sal_1c[0][:223])):
    ax4_1.plot(s_Sal_1c[1][:223,:][i,:],s_df['y'],linewidth=0.25,color=cmap(i/len(s_Sal_1c[0][:223])))
    
ax4_1.set_ylim(5,0)
ax4_1.set_xlim(34.4995,34.502)

ax4_1.yaxis.set_minor_locator(MultipleLocator(0.2))
#ax4_1.xaxis.set_minor_locator(MultipleLocator(0.25e-6))

ax4_1.set_title(r'Salinity')






m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,180)
cbar1 = plt.colorbar(m1, cax=cax, boundaries=np.linspace(0,180,len(s_S2_1c[0][:223])),ticks=[0,45,90,135,180]
                     ,pad=0.05,orientation='vertical')
cbar1.ax.set_yticklabels(['0','$\pi$/4','$\pi$/2','3$\pi$/4','$\pi$'])

ax1_1.set_ylabel('Non-Rotating')
ax1.set_ylabel('Rotating')



ax1.set_xticks([1057.806,1057.807,1057.808])
ax3.set_xticks([-2.2760,-2.2755,-2.2750])
ax4.set_xticks([34.5,34.501,34.502])

ax1_1.set_xticks([1057.806,1057.807,1057.808])
ax3_1.set_xticks([-2.2760,-2.2755,-2.2750])
ax4_1.set_xticks([34.5,34.501,34.502])

ax1.set_xticklabels(['1057.806','1057.807','1057.808'])
ax3.set_xticklabels(['-2.2760','-2.2755','-2.2750'])
ax4.set_xticklabels(['34.500','34.501','34.502'])


ax1_1.set_xticklabels([])
ax3_1.set_xticklabels([])
ax4_1.set_xticklabels([])




plt.tight_layout()
plt.show()

In [ ]:
0.0060+1.0578e3

In [ ]:
s_df

In [ ]:
s_U_1c = cycle_average_2d(s_df['U'], s_df['t'], s_df['y'], omega, 3, 6, 100)
se_U_1c = cycle_average_2d(se_df['U'], se_df['t'], se_df['y'], omega, 3, 6, 100)

In [ ]:
s_W_1c = cycle_average_2d(s_df['W'], s_df['t'], s_df['y'], omega, 3, 6, 100)
se_W_1c = cycle_average_2d(se_df['W'], se_df['t'], se_df['y'], omega, 3, 6, 100)

In [ ]:
fig = plt.figure(figsize=(5.5,5))

gs = fig.add_gridspec(4,4,width_ratios=[10,10,10,0.75])

ax1 = fig.add_subplot(gs[2:,0])
ax3 = fig.add_subplot(gs[2:,1])
ax4 = fig.add_subplot(gs[2:,2])

ax1_1 = fig.add_subplot(gs[:2,0])
ax3_1 = fig.add_subplot(gs[:2,1])
ax4_1 = fig.add_subplot(gs[:2,2])

cax = fig.add_subplot(gs[1:3,3])



cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_speed_1c[0][:223])):
    ax1.plot(se_speed_1c[1][:223,:][i,:],se_df['y'],linewidth=0.25,color=cmap(i/len(s_speed_1c[0][:223])))

ax1.set_ylim(5,0)
ax1.set_xlim(0,0.031)

ax1.yaxis.set_minor_locator(MultipleLocator(0.2))
ax1.xaxis.set_minor_locator(MultipleLocator(0.001))

#ax1.set_title(r'N$^{2}$: Stokes-Ekman')




cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_speed_1c[0][:223])):
    ax1_1.plot(s_speed_1c[1][:223,:][i,:],s_df['y'],linewidth=0.25,color=cmap(i/len(s_speed_1c[0][:223])))

ax1_1.set_ylim(5,0)
ax1_1.set_xlim(0,0.031)

ax1_1.yaxis.set_minor_locator(MultipleLocator(0.2))
ax1_1.xaxis.set_minor_locator(MultipleLocator(0.001))

ax1_1.set_title(r'Speed')







cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_U_1c[0][:223])):
    ax3.plot(se_U_1c[1][:223,:][i,:],se_df['y'],linewidth=0.25,color=cmap(i/len(s_U_1c[0][:223])))

ax3.set_ylim(5,0)
ax3.set_xlim(-0.031,0.005)

ax3.yaxis.set_minor_locator(MultipleLocator(0.2))
ax3.xaxis.set_minor_locator(MultipleLocator(0.001))

#ax3.set_title(r'TKE: Stokes-Ekman')




cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_U_1c[0][:223])):
    ax3_1.plot(s_U_1c[1][:223,:][i,:],s_df['y'],linewidth=0.25,color=cmap(i/len(s_U_1c[0][:223])))

ax3_1.set_ylim(5,0)
ax3_1.set_xlim(-0.031,0.005)

ax3_1.yaxis.set_minor_locator(MultipleLocator(0.2))
ax3_1.xaxis.set_minor_locator(MultipleLocator(0.001))

ax3_1.set_title(r'U')





cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_W_1c[0][:223])):
    ax4.plot(se_W_1c[1][:223,:][i,:],se_df['y'],linewidth=0.25,color=cmap(i/len(s_W_1c[0][:223])))

ax4.set_ylim(5,0)
ax4.set_xlim(-0.013,0.013)

ax4.yaxis.set_minor_locator(MultipleLocator(0.2))
ax4.xaxis.set_minor_locator(MultipleLocator(0.001))

#ax4.set_title(r'S$^{2}$: Stokes-Ekman')





cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_W_1c[0][:223])):
    ax4_1.plot(s_W_1c[1][:223,:][i,:],s_df['y'],linewidth=0.25,color=cmap(i/len(s_W_1c[0][:223])))
    
ax4_1.set_ylim(5,0)
ax4_1.set_xlim(-0.013,0.013)

ax4_1.yaxis.set_minor_locator(MultipleLocator(0.2))
ax4_1.xaxis.set_minor_locator(MultipleLocator(0.001))

ax4_1.set_title(r'W')






m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,180)
cbar1 = plt.colorbar(m1, cax=cax, boundaries=np.linspace(0,180,len(s_S2_1c[0][:223])),ticks=[0,45,90,135,180]
                     ,pad=0.05,orientation='vertical')
cbar1.ax.set_yticklabels(['0','$\pi$/4','$\pi$/2','3$\pi$/4','$\pi$'])

ax1_1.set_ylabel('Non-Rotating')
ax1.set_ylabel('Rotating')



ax1.set_xticks([0.00,0.01,0.02,0.03])
ax3.set_xticks([-0.03,-0.02,-0.01,0])
ax4.set_xticks([-0.01,0,0.01])

ax1_1.set_xticks([0.00,0.01,0.02,0.03])
ax3_1.set_xticks([-0.03,-0.02,-0.01,0])
ax4_1.set_xticks([-0.01,0,0.01])


ax1_1.set_xticklabels([])
ax3_1.set_xticklabels([])
ax4_1.set_xticklabels([])





plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(6.5,5))

gs = fig.add_gridspec(4,5,width_ratios=[10,10,10,10,0.75])

ax1 = fig.add_subplot(gs[2:,0])
ax3 = fig.add_subplot(gs[2:,1])
ax4 = fig.add_subplot(gs[2:,2])
ax5 = fig.add_subplot(gs[2:,3])

ax1_1 = fig.add_subplot(gs[:2,0])
ax3_1 = fig.add_subplot(gs[:2,1])
ax4_1 = fig.add_subplot(gs[:2,2])
ax5_1 = fig.add_subplot(gs[:2,3])

cax = fig.add_subplot(gs[1:3,4])



cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_u_star_1c[0][:223])):
    ax1.plot(se_u_star_1c[1][:223,:][i,:],se_df['y'],linewidth=0.25,color=cmap(i/len(s_u_star_1c[0][:223])))

ax1.set_ylim(5,0)
ax1.set_xlim(0,1.2e-3)

ax1.yaxis.set_minor_locator(MultipleLocator(0.2))
ax1.xaxis.set_minor_locator(MultipleLocator(0.1e-3))

#ax1.set_title(r'N$^{2}$: Stokes-Ekman')




cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_u_star_1c[0][:223])):
    ax1_1.plot(s_u_star_1c[1][:223,:][i,:],s_df['y'],linewidth=0.25,color=cmap(i/len(s_u_star_1c[0][:223])))

ax1_1.set_ylim(5,0)
ax1_1.set_xlim(0,1.2e-3)

ax1_1.yaxis.set_minor_locator(MultipleLocator(0.2))
ax1_1.xaxis.set_minor_locator(MultipleLocator(0.1e-3))

ax1_1.set_title(r'u*')







cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_uv_1c[0][:223])):
    ax3.plot(se_uv_1c[1][:223,:][i,:],se_df['y'],linewidth=0.25,color=cmap(i/len(s_uv_1c[0][:223])))

ax3.set_ylim(5,0)
ax3.set_xlim(-0.4e-6,1.4e-6)

ax3.yaxis.set_minor_locator(MultipleLocator(0.2))
ax3.xaxis.set_minor_locator(MultipleLocator(0.1e-6))

#ax3.set_title(r'TKE: Stokes-Ekman')




cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_uv_1c[0][:223])):
    ax3_1.plot(s_uv_1c[1][:223,:][i,:],s_df['y'],linewidth=0.25,color=cmap(i/len(s_uv_1c[0][:223])))

ax3_1.set_ylim(5,0)
ax3_1.set_xlim(-0.4e-6,1.4e-6)

ax3_1.yaxis.set_minor_locator(MultipleLocator(0.2))
ax3_1.xaxis.set_minor_locator(MultipleLocator(0.1e-6))

ax3_1.set_title(r'uv')





cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_uw_1c[0][:223])):
    ax4.plot(se_uw_1c[1][:223,:][i,:],se_df['y'],linewidth=0.25,color=cmap(i/len(s_uw_1c[0][:223])))

ax4.set_ylim(5,0)
ax4.set_xlim(-2.5e-7,2.5e-7)

ax4.yaxis.set_minor_locator(MultipleLocator(0.2))
ax4.xaxis.set_minor_locator(MultipleLocator(0.25e-6))

#ax4.set_title(r'S$^{2}$: Stokes-Ekman')





cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_uw_1c[0][:223])):
    ax4_1.plot(s_uw_1c[1][:223,:][i,:],s_df['y'],linewidth=0.25,color=cmap(i/len(s_uw_1c[0][:223])))
    
ax4_1.set_ylim(5,0)
ax4_1.set_xlim(-2.5e-7,2.5e-7)

ax4_1.yaxis.set_minor_locator(MultipleLocator(0.2))
ax4_1.xaxis.set_minor_locator(MultipleLocator(0.25e-6))

ax4_1.set_title(r'uw')






cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_wv_1c[0][:223])):
    ax5.plot(se_wv_1c[1][:223,:][i,:],se_df['y'],linewidth=0.25,color=cmap(i/len(s_wv_1c[0][:223])))

ax5.set_ylim(5,0)
ax5.set_xlim(-3.25e-7,3.25e-7)

ax5.yaxis.set_minor_locator(MultipleLocator(0.2))
ax5.xaxis.set_minor_locator(MultipleLocator(0.5e-7))

#ax5.set_title(r'R$_{g}$: Stokes-Ekman')





cmap = mpl.cm.get_cmap('rainbow')

for i in range(0,len(s_wv_1c[0][:223])):
    ax5_1.plot(s_wv_1c[1][:223,:][i,:],s_df['y'],linewidth=0.25,color=cmap(i/len(s_wv_1c[0][:223])))

ax5_1.set_ylim(5,0)
ax5_1.set_xlim(-3.25e-7,3.25e-7)

ax5_1.yaxis.set_minor_locator(MultipleLocator(0.2))
ax5_1.xaxis.set_minor_locator(MultipleLocator(0.5e-7))

ax5_1.set_title(r'wv')




m1 = plt.cm.ScalarMappable(cmap='rainbow')
m1.set_clim(0,180)
cbar1 = plt.colorbar(m1, cax=cax, boundaries=np.linspace(0,180,len(s_S2_1c[0][:223])),ticks=[0,45,90,135,180]
                     ,pad=0.05,orientation='vertical')
cbar1.ax.set_yticklabels(['0','$\pi$/4','$\pi$/2','3$\pi$/4','$\pi$'])

ax1_1.set_ylabel('Non-Rotating')
ax1.set_ylabel('Rotating')



ax1.set_xticks([0,0.0005,0.001])
ax3.set_xticks([0,0.5e-6,1e-6])
#ax4.set_xticks([-1e-6,0,1e-6])
ax5.set_xticks([-2.5e-7,0,2.5e-7])

ax1_1.set_xticks([0,0.0005,0.001])
ax3_1.set_xticks([0,0.5e-6,1e-6])
#ax4_1.set_xticks([-1e-6,0,1e-6])
ax5_1.set_xticks([-2.5e-7,0,2.5e-7])

ax1_1.set_xticklabels([])
ax3_1.set_xticklabels([])
#ax4_1.set_xticklabels([])
ax5_1.set_xticklabels([])




plt.tight_layout()
plt.show()